### Analyse concise du modèle **« Volatility Is (Mostly) Path-Dependent »**

*(Julien Guyon & Jordan Lekeufack, Quantitative Finance 2023)*

| Idée-clé                                              | Résumé                                                                                                                                                               |
| ----------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Constat empirique**                                 | Jusqu’à **90 %** de la variance du VIX s’explique par les seuls retours passés du S\&P 500. La volatilité est donc quasi totalement endogène. ([tandfonline.com][1]) |
| **Formulation continue**                              |                                                                                                                                                                      |
| Spot                                                  | $ \displaystyle \tfrac{dS_t}{S_t}= \sigma_t\,dW_t$  (sous Q)                                                                                                         |
| Vol                                                   | $ \sigma_t = \beta_0 + \beta_1 R^{(1)}_t + \beta_2\sqrt{R^{(2)}_t}$                                                                                                  |
| Mémoire — tendance                                    | $ R^{(1)}_t = \int_0^t \lambda_1 e^{-\lambda_1(t-u)}\,d\!\ln S_u$                                                                                                    |
| Mémoire — « variance »                                | $ R^{(2)}_t = \int_0^t \lambda_2 e^{-\lambda_2(t-u)} (d\!\ln S_u)^2$                                                                                                 |
| Markov 4 facteurs                                     | Deux horizons $(\lambda_{\bullet,0},\lambda_{\bullet,1})$ pour chaque intégrale → modèle « 4-FPDV » multi-échelles, Leverage + Zumbach capturés.                     |
| **Propriétés**                                        | • Modèle **complet** (un seul Brownien) → hedge Δ-only                                                                                                               |
| • Positivité de σ si $\lambda_2<2\lambda_1$.          |                                                                                                                                                                      |
| • Simulation ultra-rapide, pas de vol-de-vol imposé.  |                                                                                                                                                                      |

---

#### Implémentation (pipeline desk-production)

1. **Estimation historique (P-PDV)**

   * Retours $r_t=\ln S_t-\ln S_{t-1}$.
   * Exponentials : $R^{(1)},R^{(2)}$ via EWMA (λ en jours).
   * Régression linéaire (ou Ridge/NN) : $\sigma_{\text{cible}}$ = VIX ou RV → $(\beta_0,\beta_1,\beta_2,\lambda)$.

2. **Calibration risk-neutral (Q-PDV)**

   * Conserver la forme, ajuster $(\beta,\lambda)$ pour **surface SPX + smiles VIX** (least-squares, adjoint-MC).

3. **Monte-Carlo / PDE pricing** (Euler, Δ=1 j typ.)

   ```
   S_{t+Δ}=S_t·exp(σ_t√ΔZ−½σ_t²Δ)
   R¹_{t+Δ}=e^{-λ₁Δ}R¹_t+λ₁Δr_{t+Δ}/Δ
   R²_{t+Δ}=e^{-λ₂Δ}R²_t+λ₂Δr_{t+Δ}²/Δ
   σ_{t+Δ}=β₀+β₁R¹_{t+Δ}+β₂√R²_{t+Δ}
   ```

   * GPU-friendly, aucune Cholesky.
   * Si besoin, ajouter un petit OU « vol-de-vol » (PDSV) pour les chocs exogènes.

4. **Back-test & stress**

   * R² ≈ 0,7 pour la RV 1-5 j sur indices majeurs.&#x20;

---

#### Applications directes — **

| Cas d’usage                                            | Bénéfice PDV                                                                                              |
| ------------------------------------------------------ | --------------------------------------------------------------------------------------------------------- |
| **Exotiques equity** (autocall, cliquet, corridor-var) | Vol dépend de la trajectoire → pricing plus juste des triggers / path risk.                               |
| **Produits VIX & dispersion**                          | Calibration conjointe SPX/VIX : même modèle pour librairie delta-vol.                                     |
| **Hedge & Greeks**                                     | Modèle complet → Δ-hedge unique, vega-slippage réduit vs SV classiques.                                   |
| **Hybridisation taux/FX**                              | Vol endogène limite l’usage de Brownien supplémentaires → couplage plus stable dans les modèles hybrides. |
| **VaR / ES intraday**                                  | Simulations rapides + Zumbach/leverage capturés → scénarios réalistes.                                    |
| **Stratégies vol-target & overlay**                    | Prédiction fiable de la RV courte → ajustement dynamique de la cible de variance.                         |

---

#### Points de vigilance

* **Données** : H-F requises si λ < 1 j.
* **Stabilité** : β₁ (leverage) sensible aux régimes ; recaler régulièrement.
* **Exogène** : événements macro violents → garder composante SV résiduelle.



[1]: https://www.tandfonline.com/doi/abs/10.1080/14697688.2023.2221281?utm_source=chatgpt.com "Volatility is (mostly) path-dependent: Quantitative Finance"


## Construction détaillée du modèle PDV (« Volatility Is (Mostly) Path-Dependent », Guyon-Lekeufack)

### 1.  Intuition empirique

La variance instantanée $\sigma_t^2$ d’un indice (SPX) est expliquée **quasi entièrement** par :

1. une moyenne exponentielle des **retours signés** ($R^{(1)}$) ;
2. une moyenne exponentielle des **retours au carré** ($R^{(2)}$).

Les séries révèlent **deux horizons de mémoire** (intra-semaine vs \~1 an). D’où l’idée de superposer deux exponentielles pour chaque intégrale et d’obtenir **4 facteurs**.&#x20;

---

### 2.  Spécification continue

| Élément                     | Formule (4-FPDV)                                                                                                                                                                             |
| --------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Spot**                    | $\displaystyle\frac{dS_t}{S_t}= \sigma_t\,dW_t$                                                                                                                                              |
| **Volatilité**              | $\displaystyle \sigma_t=\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$                                                                                                                        |
| **Kernels**                 | $K_{n}(\tau)=(1-\theta_n)\lambda_{n,0}e^{-\lambda_{n,0}\tau}+\theta_n\lambda_{n,1}e^{-\lambda_{n,1}\tau}$                                                                                    |
| **Facteurs**                | $R_{n,t}= (1-\theta_n)R_{n,0,t}+\theta_n R_{n,1,t}$                                                                                                                                          |
| **Dynamiques Markoviennes** | $\displaystyle\begin{cases} dR_{1,j,t}= \lambda_{1,j}\bigl(\sigma_t\,dW_t-R_{1,j,t}\,dt\bigr)\\ dR_{2,j,t}= \lambda_{2,j}\bigl(\sigma_t^{2}-R_{2,j,t}\bigr)dt \end{cases}\qquad j\in\{0,1\}$ |

**Dimensions :** 5D Markov ($S$ + 4 facteurs). Non explosion et unicité fortes démontrées pour toutes valeurs de paramètres. ([arxiv.org][1])

*Conditions pratiques :* $\beta_0,\beta_2,\lambda_{n,j}\ge0$, $\beta_1\le0$ (effet leverage), et pour garantir $\sigma_t\ge0$ on prend souvent $\lambda_{2,\bullet}<2\lambda_{1,\bullet}$.

---

### 3.  Paramètres et sens économique

| Paramètre         | Impact                                             |
| ----------------- | -------------------------------------------------- |
| $\beta_0$         | niveau de base (vol plancher)                      |
| $\beta_1<0$       | **Leverage** : chute du spot ↑ vol                 |
| $\beta_2>0$       | convexité (cluster de volatilité)                  |
| $\lambda_{1,0/1}$ | vitesses de **mémoire sur retours** (jours ↔ mois) |
| $\lambda_{2,0/1}$ | vitesses de **mémoire sur var. passée**            |
| $\theta_{1,2}$    | pondération long / court terme                     |

---

### 4.  Implémentation numérique (lib pricers desk)

#### 4.1  Pré-calcul des facteurs (back-filling)

En temps discret $t_k=k\Delta$ (Δ = 1 jour ou 5 min) :

```python
R1_j[k] = exp(-λ1_j Δ)*R1_j[k-1] + λ1_j * r[k]
R2_j[k] = exp(-λ2_j Δ)*R2_j[k-1] + λ2_j * r[k]**2
σ[k]    = β0 + β1*R1[k] + β2*sqrt(R2[k])
```

Processus **O(1)**, parallélisable GPU.

#### 4.2  Simulation / pricing

Boucle Euler-lognormale :

$$
S_{k+1}=S_k\exp\!\bigl(\sigma_k\sqrt{\Delta}Z-\tfrac12\sigma_k^2\Delta\bigr)
$$

puis mise-à-jour des quatre $R_{n,j}$ comme ci-dessus. Stable pour Δ ≤ 1 j.

#### 4.3  Calibration Q (« surface jointe » SPX + VIX)

1. **Initial guess :** λ fixés à ceux estimés sous P, β ajustés.
2. **Loss** = somme pondérée des erreurs prix SPX + VIX futures + VIX options.
3. **Optimisation** (Py-BOBYQA / CMA-ES) sur \~50 k scénarios GPU ⇒ < 30 min pour 10 paramètres. ([arxiv.org][2])
4. **Greeks** via adjoint/pathwise, donc ∂/∂β, ∂/∂λ gratuits.

---

### 5.  Extensions utiles 

| Besoin                     | Ajustement                                                                                    |
| -------------------------- | --------------------------------------------------------------------------------------------- |
| **Sauts macro**            | Ajouter un jump Poisson sur $S_t$ ou directement un choc additif sur $\sigma_t$.              |
| **Cross-asset (rates/FX)** | Utiliser même Brownien $W_t$ couplé à Heston-IR ou log-normal FX pour garder dimension basse. |
| **Stress scripts**         | Forcer β₁ → β₁\*(1+ϵ) pour tester scenarios leverage extrême.                                 |

---

### 6.  Pipeline « prod » 

1. **Data feed** haute fréquence → rolling EWMAs pour tenir à jour $R_{n,j}$.
2. **Job nightly** : re-estimation β, recalibration Q (GPU farm).
3. **Cache** surfaces *SPX/VIX exotiques* + *vol-target* prêtes à 7 h UK.
4. **Real-time risk** : delta unique (complet) ; dérivées CVA via bump-&-reprice (simulation << Heston).

---

### 7.  Points de vigilance

* **Stabilité β₁** : très sensible aux régimes, surveiller breakpoints macro.
* **Δ trop grand** → biais (notamment sur VIX 1m). Garder Δ ≤ 0,2 j pour equity vol surfaces.
* **Positivité** : vérifier $\sigma_t>0$ à chaque pas ; sinon clip minimal (ε ≈ 0,1 vol).

---

**À retenir :** le 4-FPDV convertit une dynamique de volatilité fortement empirique (mémoire courte + longue, leverage, clustering) en un système Markovien compact et calculable, idéal pour **pricing rapide**, **calibration jointe** et **hedging simple** .

[1]: https://arxiv.org/pdf/2307.01319 "On the Guyon–Lekeufack Volatility Model"
[2]: https://arxiv.org/html/2406.02319v1 "Pricing and calibration in the 4-factor path-dependent volatility model"


Empiriquement :

1. **Effet court terme (intra-semaine)**
   L’autocorrélation des retours signés et de la variance passée décroît très vite : après ≈ 3-5 jours elle est déjà quasi nulle.
   ⇒ mémoire « rapide » ≈ quelques jours.

2. **Effet long terme (\~ 1 an)**
   Au-delà, on observe une traîne faible mais persistante : la corrélation reste > 0 sur plusieurs centaines de jours.
   ⇒ mémoire « lente » ≈ 250-300 jours.

Pour représenter **les deux échelles simultanément** on prend, pour chaque noyau de mémoire,

$$
K(\tau)= (1-\theta)\,\lambda_0 e^{-\lambda_0 \tau}\;+\;\theta\,\lambda_1 e^{-\lambda_1 \tau},
$$

*superposition de deux exponentielles* :

* $\lambda_0^{-1}$ calibre la mémoire rapide,
* $\lambda_1^{-1}$ la mémoire lente,
* $\theta$ règle leur poids relatif.

On applique ce mix à :

* $R^{(1)}$ (moyenne exponentielle des **retours signés**)
* $R^{(2)}$ (moyenne exponentielle des **retours au carré**)

→ chaque intégrale se scinde en **deux composantes** (court + long terme).
Au total : $R^{(1)}_{\text{court}}, R^{(1)}_{\text{long}}, R^{(2)}_{\text{court}}, R^{(2)}_{\text{long}}$ — soit **4 facteurs** dans la version Markovienne du modèle.


### « Retour signé » vs « retour au carré »

| Terme                          | Définition mathématique                                                                                                                      | Rôle économique                                                                                                                               |
| ------------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------- |
| **Retour signé** $r_t$         | $r_t=\ln\!\bigl(S_t/S_{t-\Delta}\bigr)$  (ou $S_t/S_{t-\Delta}-1$) — il garde son **signe** : positif si le spot monte, négatif s’il baisse. | Porte l’**information directionnelle** : un retour négatif (chute du marché) tend à faire monter la vol (effet leverage).                     |
| **Retour au carré** $r_t^{ 2}$ | $(\ln(S_t/S_{t-\Delta}))^{2}$                                                                                                                | Mesure la **magnitude** du mouvement sans tenir compte du signe ; c’est l’élément de base de la variance réalisée (clustering de volatilité). |

Dans le modèle PDV :

* $R^{(1)}_t$ = moyenne exponentielle des **retours signés** → encode l’effet leverage (β₁ < 0).
* $R^{(2)}_t$ = moyenne exponentielle des **retours au carré** → encode le **niveau** de volatilité passée (β₂ > 0).

Ainsi, la combinaison $\sigma_t = β_0 + β_1 R^{(1)}_t + β_2 \sqrt{R^{(2)}_t}$ capte à la fois la **direction récente** du marché et l’**intensité** de ses fluctuations.


### Définitions précises de $R^{(1)}$ et $R^{(2)}$

#### 1. En temps continu (version 4-facteurs)

$$
\boxed{
\begin{aligned}
R^{(1)}_t &\;=\;(1-\theta_1)R^{(1)}_{\text{court},t} + \theta_1 R^{(1)}_{\text{long},t},\\[3pt]
R^{(2)}_t &\;=\;(1-\theta_2)R^{(2)}_{\text{court},t} + \theta_2 R^{(2)}_{\text{long},t},
\end{aligned}}
$$

avec, pour chaque horizon $j\in\{\text{court},\text{long}\}$,

$$
\boxed{
\begin{aligned}
R^{(1)}_{j,t} &= \int_{0}^{t} \lambda_{1,j}\,e^{-\lambda_{1,j}(t-u)}\,d\!\ln S_u, \\[6pt]
R^{(2)}_{j,t} &= \int_{0}^{t} \lambda_{2,j}\,e^{-\lambda_{2,j}(t-u)}\,\bigl(d\!\ln S_u\bigr)^2.
\end{aligned}}
$$

* **$R^{(1)}$** : **moyenne exponentielle des retours signés** (direction du marché).
* **$R^{(2)}$** : **moyenne exponentielle des retours au carré** (intensité des mouvements).

Chaque intégrale utilise **deux constantes de décroissance**
— $\lambda_{\bullet,\text{court}}^{-1}$ ≈ quelques jours,
— $\lambda_{\bullet,\text{long}}^{-1}$ ≈ un an —
pour refléter les deux horizons de mémoire observés.

---

#### 2. En temps discret (pas $\Delta$, typiquement 1 jour ou 5 min)

Pour éviter de ré-intégrer toute l’historique, on met les intégrales sous forme **récurrente (O(1))** :

$$
\boxed{
\begin{aligned}
R^{(1)}_{j,k} &= e^{-\lambda_{1,j}\Delta}\,R^{(1)}_{j,k-1} + \lambda_{1,j}\,r_k,\\[4pt]
R^{(2)}_{j,k} &= e^{-\lambda_{2,j}\Delta}\,R^{(2)}_{j,k-1} + \lambda_{2,j}\,r_k^{2},
\end{aligned}}
$$

où $r_k = \ln(S_k/S_{k-1})$.

On en déduit la volatilité instantanée :

$$
\boxed{\;\sigma_k \;=\; \beta_0 + \beta_1 R^{(1)}_k + \beta_2 \sqrt{R^{(2)}_k}\;}
$$

---

#### 3. Interprétation rapide

| Terme     | Sens                                                                      | Influence sur $\sigma_t$                                    |
| --------- | ------------------------------------------------------------------------- | ----------------------------------------------------------- |
| $R^{(1)}$ | **“Mémoire directionnelle”** : somme pondérée de tous les retours, signés | $\beta_1<0$  ⇒ baisse récente → vol ↑ (leverage)            |
| $R^{(2)}$ | **“Mémoire de variance”** : somme pondérée de tous les retours au carré   | $\beta_2>0$  ⇒ gros mouvements récents → vol ↑ (clustering) |

Ainsi, $R^{(1)}$ capte l’asymétrie (leverage), tandis que $R^{(2)}$ capte le niveau de volatilité ; leur décomposition « court / long terme » donne les **4 facteurs** du modèle Markovien.


Exact !

* **Directionnel court terme ($R^{(1)}_{\text{court}}$)**
  *Mémoire rapide* des retours signés (demi-vie ≈ 2-5 j) → capte le **leverage** intra-semaine.

* **Directionnel long terme ($R^{(1)}_{\text{long}}$)**
  *Mémoire lente* des retours signés (demi-vie ≈ 6-12 mois) → reflète la pente « bear-ish » de long terme.

* **Niveau (variance) court terme ($R^{(2)}_{\text{court}}$)**
  Moyenne exponentielle des $r^2$ à horizon jours → décrit le **clustering** très court de la vol.

* **Niveau (variance) long terme ($R^{(2)}_{\text{long}}$)**
  Moyenne exponentielle des $r^2$ sur \~1 an → fixe la **régime-vol** (bas, moyen, crise).

Ces quatre processus Markoviens suivent :

$$
\begin{aligned}
dR^{(1)}_{j,t}&=\lambda_{1,j}\!\left(\sigma_t\,dW_t-R^{(1)}_{j,t}\,dt\right),\\
dR^{(2)}_{j,t}&=\lambda_{2,j}\!\left(\sigma_t^{2}-R^{(2)}_{j,t}\right)dt,\qquad j\in\{\text{court},\text{long}\},
\end{aligned}
$$

et la volatilité instantanée est simplement

$$
\sigma_t=\beta_0+\beta_1\bigl[(1-\theta_1)R^{(1)}_{\text{court}}+\theta_1R^{(1)}_{\text{long}}\bigr]
          +\beta_2\sqrt{(1-\theta_2)R^{(2)}_{\text{court}}+\theta_2R^{(2)}_{\text{long}} }.
$$

> **En résumé** :
> • 2 horizons × 2 types de mémoire = **4 facteurs**
> • Les deux $R^{(1)}$ gouvernent l’effet *leverage* (signe),
> • Les deux $R^{(2)}$ gouvernent le *level* de volatilité (amplitude).


### Développement : passage **continu → discret** pour les 4 facteurs

---

#### 1.  Facteur directionnel (exemple court-terme)

Continuous form

$$
R^{(1)}_{c}(t)=\int_{0}^{t}\lambda_{1c}\,e^{-\lambda_{1c}(t-u)}\;d\ln S_u .
\tag{1}
$$

---

##### 1.1  SDE exacte

En dérivant (1) :

$$
\frac{d}{dt}R^{(1)}_{c}(t)=\lambda_{1c}\,d\ln S_t-\lambda_{1c} R^{(1)}_{c}(t)\,dt
\quad\Longrightarrow\quad
dR^{(1)}_{c}= \lambda_{1c}\,d\ln S_t-\lambda_{1c}R^{(1)}_{c}\,dt .
\tag{2}
$$

---

##### 1.2  Solution fermée entre deux dates $t_k,\;t_{k+1}=t_k+\Delta$

L’ODE linéaire (2) donne

$$
R^{(1)}_{c}(t_{k+1}) =
e^{-\lambda_{1c}\Delta}\,R^{(1)}_{c}(t_k) + \lambda_{1c}
\int_{t_k}^{t_{k+1}}e^{-\lambda_{1c}(t_{k+1}-u)}d\ln S_u .
\tag{3}
$$

---

##### 1.3  Recursion pratique

Sur le pas $\Delta$ on note le **log-retour**
$r_{k+1}=\ln S_{k+1}-\ln S_k$.

En supposant que $d\ln S_u$ est (quasi) constant sur l’intervalle :

$$
\int_{t_k}^{t_{k+1}}e^{-\lambda_{1c}(t_{k+1}-u)}d\ln S_u
\;\approx\;
\underbrace{\frac{1-e^{-\lambda_{1c}\Delta}}{\Delta}}_{\beta_{1c}}
\,r_{k+1}.
$$

On obtient la mise à jour **exacte au premier ordre**

$$
\boxed{\,R^{(1)}_{c,k+1}
    =\alpha_{1c}\,R^{(1)}_{c,k}+\beta_{1c}\,r_{k+1}\,},
\qquad
\alpha_{1c}=e^{-\lambda_{1c}\Delta},\;
\beta_{1c}=1-\alpha_{1c}.
\tag{4}
$$

> Dans le code on voit souvent la forme simplifiée
> $\;R^{(1)}_{c,k+1}=e^{-\lambda_{1c}\Delta}R^{(1)}_{c,k}+\lambda_{1c}r_{k+1}$ ;
> elle revient à remplacer $\beta$ par $\lambda\Delta$ (erreur $\mathcal O(\Delta^2)$ négligeable pour $\Delta\le1$ j).

---

#### 2.  Facteur de niveau (variance)

Continu :

$$
R^{(2)}_{c}(t)=\int_0^t\lambda_{2c}e^{-\lambda_{2c}(t-u)}\sigma_u^{2}\,du .
\tag{5}
$$

On obtient l’ODE
$dR^{(2)}_{c}= \lambda_{2c}(\sigma_t^{2}-R^{(2)}_{c})\,dt$.

Entre deux dates :

$$
R^{(2)}_{c,k+1}=e^{-\lambda_{2c}\Delta}R^{(2)}_{c,k}
   +(1-e^{-\lambda_{2c}\Delta})\,\sigma_{k}^{2}.
\tag{6}
$$

*Si l’on travaille sur données historiques on remplace*
$\sigma_{k}^{2}\approx r_{k+1}^{2}/\Delta$.

---

#### 3.  Même logique pour les horizons **long terme**

On répète (4)–(6) avec $\lambda_{1\ell}$ et $\lambda_{2\ell}$.

---

### 4.  Algorithme final (4 facteurs)

| Mise à jour (pas $\Delta$)                                                                                     |
| -------------------------------------------------------------------------------------------------------------- |
| $R^{(1)}_{\text{court},k+1}= \alpha_{1c} R^{(1)}_{\text{court},k} + \beta_{1c} r_{k+1}$                        |
| $R^{(1)}_{\text{long },k+1}= \alpha_{1\ell}R^{(1)}_{\text{long },k}+ \beta_{1\ell}r_{k+1}$                     |
| $R^{(2)}_{\text{court},k+1}= \alpha_{2c} R^{(2)}_{\text{court},k}+ (1-\alpha_{2c})\,\hat{\sigma}_{k}^{2}$      |
| $R^{(2)}_{\text{long },k+1}= \alpha_{2\ell}R^{(2)}_{\text{long },k}+ (1-\alpha_{2\ell})\,\hat{\sigma}_{k}^{2}$ |

avec
$\alpha_{\bullet}=e^{-\lambda_{\bullet}\Delta},\;
\beta_{1\bullet}=1-\alpha_{1\bullet},\;
\hat{\sigma}_{k}^{2}=r_{k+1}^{2}/\Delta$ (hist) ou $\sigma_{k}^{2}$ (simu).

La volatilité instantanée utilisée dans le pricing est alors

$$
\sigma_k=\beta_0+\beta_1\bigl[(1-\theta_1)R^{(1)}_{\text{court},k}+\theta_1R^{(1)}_{\text{long},k}\bigr]
      +\beta_2\sqrt{(1-\theta_2)R^{(2)}_{\text{court},k}+\theta_2R^{(2)}_{\text{long},k}}.
$$

---

**En bref :** chacune des quatre exponentielles est **exactement** recalculée en $O(1)$ grâce aux coefficients $\alpha=e^{-\lambda\Delta}$ et $\beta=1-\alpha$. Cette discrétisation est cohérente avec la solution fermée des ODEs, donc stable et sans biais à l’ordre 1 pour $\Delta\le1$ jour.


### Que sont les $\lambda$ ?

*Coefficients de décroissance* des mémoires exponentielles :

| Symbole            | Mémoire                            | Horizon caractéristique                     | Interprétation économique                                                  |
| ------------------ | ---------------------------------- | ------------------------------------------- | -------------------------------------------------------------------------- |
| $\lambda_{1c}$     | retours **signés** – court terme   | $T_{½}\!\approx\!\ln2/\lambda_{1c}$ = 2-5 j | Rapidité d’apparition / disparition de l’effet **leverage** intraday-hebdo |
| $ \lambda_{1\ell}$ | retours **signés** – long terme    | 6-12 mois                                   | Persistance d’un biais baissier de fond                                    |
| $\lambda_{2c}$     | retours **au carré** – court terme | 1-10 j                                      | Durée des bouffées de vol (vol-spikes)                                     |
| $ \lambda_{2\ell}$ | retours **au carré** – long terme  | 6-18 mois                                   | Vitesse de retour au régime-vol « normal » après crise                     |

*(Le demi-vie $T_{½}$ est souvent plus parlant que $1/\lambda$.*)

---

### Comment les estimer ?  — **Sous P (historique)**

1. **Recherche brutale ou ML léger**

   * Balayer une grille de paires $(\lambda_{1c},\lambda_{1\ell})$ et $(\lambda_{2c},\lambda_{2\ell})$.
   * Pour chaque quadriplet : calculez $R^{(1)}$ & $R^{(2)}$, réglez $(\beta_0,\beta_1,\beta_2)$ par régression sur la RV future (ou VIX).
   * Garder le jeu qui maximise $R^2$ hors échantillon.

2. **Méthode « auto-corrélation cible »**

   * Mesurer la fonction d’auto-corrélation empirique de $|r_t|$ et de $r_t^2$.
   * Ajuster deux exponentielles de la forme $a e^{-\lambda_c \tau} + (1-a) e^{-\lambda_\ell \tau}$ en moindres carrés → donne directement les quatre $\lambda$.

3. **Kalman / EM** (si l’on estime les facteurs latent et les paramètres simultanément).

---

### Calibration **sous Q** (optionnelle)

* Commencez avec les $\lambda$ historiques.
* Recalibrez-les en bloc (ou seulement $\lambda_{2c}$ & $\lambda_{2\ell}$) sur la **surface SPX + VIX** ; ils pilotent la pente temporelle de la vol implicite.
* Le plus souvent on touche peu aux $\lambda_1$ (leverage) et on ajuste surtout $\beta$.

---

### Lecture rapide

* **Petit $\lambda$ ⇒ longue mémoire** : le facteur bouge lentement, stabilise la vol de long terme.
* **Grand $\lambda$ ⇒ mémoire courte** : facteur très réactif, nécessaire pour reproduire les pics intraday/journaliers.
* **Ratio $\lambda_{2}/\lambda_{1}$** (> 1 typiquement) assure la positivité de $\sigma_t$.
* Sensibilité : les smiles lointains dépendent surtout de $\lambda_{2\ell}$; les options courtes d’échéance dépendent de $\lambda_{2c}$.

En pratique, une fois les $\lambda$ fixés, le modèle devient **linéaire** en $(\beta_0,\beta_1,\beta_2)$ et se recalcule en quelques millisecondes — d’où la robustesse de la calibration quotidienne


### Comment les $\lambda$ **codifient** la mémoire court / long terme

*(et pourquoi il en faut exactement quatre)*

| Symbole  | Horizon « 1/ λ » | Demi-vie $T_{½}= \ln2/λ$ | Effet concret dans le modèle                                                                           |
| -------- | ---------------- | ------------------------ | ------------------------------------------------------------------------------------------------------ |
| $λ_{1c}$ | 1-10 jours       | 2-5 j                    | **Leverage flash** : la vol bondit quand le spot chute brusquement, puis s’oublie en quelques séances. |
| $λ_{1ℓ}$ | 6-12 mois        | 100-250 j                | **Biais directionnel lent** : tendance baissière durable ↗ vol implicite de long terme.                |
| $λ_{2c}$ | 1-15 jours       | 3-7 j                    | **Clustering court** : un spike de vol (CPI surprise…) se résorbe en \~ une semaine.                   |
| $λ_{2ℓ}$ | 6-18 mois        | 150-300 j                | **Régime de volatilité** : sortie de crise 2008/2020, retour au niveau de croisière.                   |

---

#### 1.  Du noyau exponentiel à la **mémoire effective**

Une moyenne exponentielle

$$
K_{λ}(\tau)=λe^{-λ\tau}\;\;(τ\ge0)
$$

attribue au retour vieux de $τ$ jours le poids
$w(τ)=e^{-λτ}$.

* **Grand $λ$** ⇒ pente raide ⇒ ne « voit » que le passé proche.
* **Petit $λ$** ⇒ pente molle ⇒ garde trace d’événements anciens.

---

#### 2.  Superposition : 2 horizons × 2 types

$$
\text{Poids total}(τ)=
(1-\theta)\,λ_{c}e^{-λ_{c}τ}
+\theta\,λ_{ℓ}e^{-λ_{ℓ}τ}.
$$

* **$\theta$** règle « le curseur » entre court (0) et long terme (1).
* La **courbe** de poids épouse très bien l’autocorrélation empirique, creusée à l’origine et traînante plus loin.

---

#### 3.  Implémentation **discrète** : 1 ligne de code par facteur

```python
α = exp(-λ * Δ)      # mémoire résiduelle
β = 1 - α            # poids du retour courant
R_next = α * R + β * r             # pour les retours signés
V_next = α * V + (1-α) * sigma**2  # pour les retours²
```

Choisissez $λ$ :

| Cible empirique     | Recette rapide                     |
| ------------------- | ---------------------------------- |
| Spike intraday      | $λ_{2c}= \ln2 / 3\text{ j}^{-1}$   |
| Bridge 3-6 m        | $λ_{2ℓ}= \ln2 / 180\text{ j}^{-1}$ |
| Leverage day-to-day | $λ_{1c}= \ln2 / 2\text{ j}^{-1}$   |
| Leverage macro      | $λ_{1ℓ}= \ln2 / 120\text{ j}^{-1}$ |

---

#### 4.  Impact direct sur les **smiles** et la **courbe de vol**

| Paramètre  | Sourires courts           | Sourires longs                | Term-structure VIX               |
| ---------- | ------------------------- | ----------------------------- | -------------------------------- |
| $λ_{2c}$ ↑ | smiles 1-30 d plus pentus | quasi nul                     | élève VIX front-month            |
| $λ_{2ℓ}$ ↓ | peu d’effet               | creuse / twist smiles 1-3 ans | relève VIX 1-2 ans               |
| $λ_{1c}$ ↑ | accentue skew très court  | nul                           | baisse/relève VIX selon signe β₁ |
| $λ_{1ℓ}$ ↓ | lisse skew court          | accentue skew long            | hausse vol longue                |

---

#### 5.  Pourquoi **quatre** est suffisant

1. Mémoire empirique = **double-exponentielle** → 2 paramètres par type.
2. Deux types (direction vs niveau) → 4 facteurs.
3. Ajout d’un troisième horizon → gain minime, coût CPU + calibration ↑.

---

**À retenir** :

* Les $λ$ ne sont pas « des » vitesses : ce sont *les* boutons qui transforment les retours bruts en une **mémoire pondérée** parfaitement réglable.
* En jouant sur leurs valeurs (et sur $\theta$), on **calibre** la réactivité ultra-courte et la persistance longue durée **sans ajouter de Vol-de-Vol**.
* Un desk n’a qu’à figer les $λ$ (court / long) une fois, puis recalibrer chaque jour les $\beta$ — cycle robuste et rapide.


### Formule « forward-variance » → pont direct entre **SPX** et **VIX**

Pour un horizon $T$ (30 j pour le VIX spot), le futur VIX est

$$
\boxed{F^{\text{VIX}}_0(T)=100
\sqrt{\frac{1}{T}\;
\underbrace{\mathbb E_Q\!\Bigl[\int_{0}^{T}\!\sigma_t^{2}\,dt\Bigr]}_{:=\,\Xi(T)}}}\;.
$$

Dans le 4-FPDV, les deux facteurs de niveau $R^{(2)}_{c},R^{(2)}_{\ell}$ sont des processus OU :

$$
dR^{(2)}_{j,t}= \lambda_{2j}\bigl(\sigma_t^{2}-R^{(2)}_{j,t}\bigr)dt,\qquad j\in\{c,\ell\}.
$$

En prenant l’espérance conditionnelle (Markov, mémoire exponentielle) on obtient exactement :

$$
\Xi(T)=A_0(T)+A_c(T)\,R^{(2)}_{c}(0)+A_\ell(T)\,R^{(2)}_{\ell}(0),
$$

avec
$A_j(T)=\dfrac{1-e^{-\lambda_{2j}T}}{\lambda_{2j}}$(pondérés par $\theta_2$).
**⇒ VIX futur** (et donc les options VIX) se lit **linéairement** dans l’état courant des deux $R^{(2)}$.

---

## Routine de calibration conjointe **SPX + VIX**

> Objectif : trouver $(\beta_0,\beta_1,\beta_2)$ (et éventuellement $λ_{2c},λ_{2\ell}$) qui minimisent
> l’erreur prix sur *SPX vanilles* **et** *futurs/op­tions VIX*.

| Étape                 | Détail pratique                                                                                                                                                                                                                                  |
| --------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **1. Fixer les λ**    | Prenez ceux estimés sous P (ou ré-optimisez $λ_{2\bullet}$ uniquement ; toucher à $λ_{1\bullet}$ casse le skew).                                                                                                                                 |
| **2. État courant**   | Calculez $R^{(2)}_{c}(0),R^{(2)}_{\ell}(0)$ à partir des retours des 12 derniers mois (recursion O(1)).                                                                                                                                          |
| **3. Seed linéaire**  | La formule analytique de $\Xi(T)$ rend le problème **linéaire en β** → une régression GLS sur les prix **VIX futures** fournit un premier $(β_0,β_2)$. Fixez provisoirement $β_1$ avec le skew SPX 1 M.                                          |
| **4. Objectif joint** | $\text{Loss}=w_{\text{SPX}}\sum(\text{IV}^{\text{mod}}-\text{IV}^{\text{mkt}})^2 + w_{\text{VIX}}\sum(\Xi^{\text{mod}}-\Xi^{\text{mkt}})^2.$ <br>• SPX : prix via MC (≈ 10 k chemins, GPU) <br>• VIX futures : formule fermée ci-dessus (0 cpu). |
| **5. Optimisation**   | CMA-ES global puis L-BFGS local (10 paramères → < 30 s sur GPU). Tous les gradients sont automatiques ; $\partial \Xi/\partial β$ est analytique.                                                                                                |
| **6. VIX options**    | Prix avec un petit MC **antithétique** où l’on recycle la valeur fermée de $\Xi(T)$ comme control-variate ⇒ erreur<0.3 vol-pt. (On les **laisse dans la fonction de coût** pour verrouiller curvature).                                          |
| **7. Validation**     | < 1 vol-pt RMS sur SPX δ 20-80 % & VIX 1-24 m ; R² > 70 % sur RV\_5d hors-échantillon.                                                                                                                                                           |

---

### Pourquoi la formule ferme accélère vraiment

* Sans elle : il faudrait simuler $\int\sigma_t^2dt$ dans la boucle calib → ×100 plus lent.
* Avec elle : **VIX futures = simple racine carrée d’un polynôme affine** des deux $R^{(2)}$ courants ; les dérivées $\partial/\partial β,λ$ tombent en une ligne.
* Résultat : joint-fit complet SPX+VIX en routine nocturne < 2 min, greeks disponibles en temps réel pour XVA/risk.

---

#### Formule flash (prête à coder)

```python
def vix_future(R2c0, R2l0, beta0, beta2, lam2c, lam2l, theta2, T):
    Ac = (1-np.exp(-lam2c*T))/lam2c
    Al = (1-np.exp(-lam2l*T))/lam2l
    Xi = beta0**2*T + beta2**2*((1-theta2)*Ac*R2c0 + theta2*Al*R2l0)
    return 100*np.sqrt(Xi/T)
```

> **À retenir** :
>
> * La *forward variance* se résume à trois coefficients $(A_0,A_c,A_\ell)$ déterminés par les seuls $λ_{2\bullet}$.
> * Cette linéarité rend la calibration **rapide, stable et parfaitement cohérente** entre SPX et VIX.


**σ\_t dans le modèle PDV = volatilité *instantanée* (spot)**

Voici la version corrigée du petit tableau — sans « trou » — qui situe clairement **σₜ** par rapport aux autres notions :

| Quantité                            | Formule / dynamique                                                               | Nature de la mesure                                                                                            | Utilisation principale                                           |
| ----------------------------------- | --------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------- |
| **σₜ**                              | $\displaystyle \sigma_t = \beta_0 + \beta_1 R^{(1)}_t + \beta_2 \sqrt{R^{(2)}_t}$ | Volatilité **instantanée (spot)**, définie dans le modèle (sous P pour l’estimation, sous Q après calibration) | Simuler $S_t$ via $dS_t/S_t = σ_t\,dW_t$ → base pour RV, IV, VIX |
| **Volatilité réalisée** sur $[0,T]$ | $\sqrt{\tfrac1T \int_0^T σ_u^{2}\,du}$                                            | Mesure **ex post**                                                                                             | Back-tests, contrôle de qualité du modèle                        |
| **Forward variance** $ \Xi(T)$      | $\mathbb E_Q\!\bigl[ \int_0^T σ_u^{2} du \bigr]$                                  | Variance future **sous Q** (donc implicite)                                                                    | Entre directement dans la quote des **futures / options VIX**    |
| **Volatilité implicite (IV)**       | Valeur de σ telle que Black-Scholes reproduit le prix option                      | Objet de marché, dérivée de la loi future de $S_T$                                                             | Surface SPX, calibration quotidienne                             |

> **À retenir** : la formule de **σₜ** décrit la vol instantanée ; toutes les autres volatilités (réalisée, forward / VIX, implicite) sont des agrégats ou des fonctions de ce σₜ.

* **Sous P** (historique) quand on estime le modèle sur les retours passés ;
* **Sous Q** (risk-neutral) après calibration SPX + VIX ; | Sert à simuler $S_t$ via $dS_t/S_t=\sigma_t\,dW_t$ et donc à **produire** :<br>• la volatilité réalisée future ;<br>• les surfaces d’IV (via espérance sous Q). |

---

### Comment relier σ\_t aux autres notions

| Notion                                                | Formule à partir de σ\_t                                                                       | Commentaire                                                                        |
| ----------------------------------------------------- | ---------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------- |
| **Volatilité réalisée** sur \[0,T]                    | $\displaystyle \sqrt{\frac1T\int_0^T \sigma_u^{2}\,du}$                                        | Connue **a-posteriori** ; utilisée pour back‐tests.                                |
| **Forward variance** (clé du VIX)                     | $\displaystyle \Xi(T)=\mathbb E_Q\!\Bigl[\int_0^{T}\sigma_u^{2}\,du\Bigr]$                     | Se calcule analytiquement dans PDV (linéaire en $R^{(2)}$).                        |
| **Volatilité impliquée (IV)** d’une option maturité T | Trouvée numériquement : prix option = $E_Q[\text{payoff}(S_T)]$ où $S_t$ suit $dS_t$ ci-dessus | L’IV est une **fonction** de la loi future de $σ_t$ ; ce n’est pas $σ_t$ lui-même. |

---

**En résumé :**

* La formule que je t’ai donnée décrit l’**état instantané** de la volatilité dans le modèle.
* Une fois σ\_t simulé (ou son espérance intégrée) on en déduit :

  * **réalisée** : intégrale de σ\_t² sur la période,
  * **impliquée / VIX** : espérance sous Q de cette même intégrale.
* Donc σ\_t n’est ni l’IV ni la RV ; c’est la *matière première* dont les deux découlent.


### Market data indispensables pour un **setup SPX + VIX**

*(tout ce qui est vraiment nécessaire pour estimer les paramètres sous P, calibrer sous Q, back-tester et produire les grecs)*

| Catégorie                                 | Détail précis                                                                                                                                                                                       | Fréquence / profondeur minimale                                      |
| ----------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------- |
| **1. Sous-jacent SPX**                    | • Spot index $S_0$<br>• Historique de clôture (≥10 ans, pas = 1j)<br>• Ajustements dividendes (cash & add-on)                                                                                       | daily (intraday si back-test haute fréquence)                        |
| **2. Options SPX (vanilles)**             | Pour chaque date : <br>• Liste strikes, maturités, bid/ask, volume, open interest<br>• Implied vol $\sigma_{\text{mkt}}$ ou prix ($C,P$)<br>• Type américain/européen (SPX = euro), style règlement | all maturities : 1 sem ↔ 5 ans<br>granularité ≈ 5–10 strikes / smile |
| **3. Courbe taux sans risque**            | • OIS / SOFR zero-curve pour actualisation<br>• Fed-funds futures (si modeling forward vols)<br>• Dividend yield term-structure du S\&P 500                                                         | daily                                                                |
| **4. VIX spot**                           | • Cours officiel VIX (t-close)                                                                                                                                                                      | daily                                                                |
| **5. Term structure VIX futures**         | • Prix futures (VX1,…VX13), dates d’échéance<br>• Bid/ask & volume                                                                                                                                  | daily                                                                |
| **6. Options VIX**                        | • Strikes, maturités, bid/ask, implied vol<br>• Volume & open interest                                                                                                                              | maturités 1–6 mois, strikes 80–120 %                                 |
| **7. “Strip” OTM SPX pour répliquer VIX** | • Quotations OTM (5 %–50 % ITM/OTM) même échéance 30 j                                                                                                                                              | même fréquence que VIX futures (au moins daily)                      |
| **8. Réalisé (realized) volatility**      | • Série 30-day RV (close-to-close ou intraday-5 min) pour validation P                                                                                                                              | daily                                                                |
| **9. Calendrier de marché**               | • Jours ouvrés, jours fériés, heures de clôture                                                                                                                                                     | statique + maj annuelle                                              |
| **10. Métadonnées qualité/liquidité**     | • Spreads, vega, theta, intervalles de confiance des IV                                                                                                                                             | optionnel mais recommandé pour pondérations de coût                  |

> **Récap express** :
>
> * Pour **P-measure** : historiques hauteurs SPX + RV.
> * Pour **Q-measure / calibration** : surfaces d’options SPX & VIX, strip OTM pour la formule VIX, courbe taux/dividende.
> * Pour **validation / stress** : futures VIX & realised vol.

Avec ces dix blocs, tu couvres **100 %** des besoins données du modèle “Vol Mostly Path-Dependent” appliqué conjointement à SPX et VIX.


### Différence clé entre **mesure historique $\mathbb P$** et **mesure risque-neutre $\mathbb Q$** dans le modèle “Vol Mostly Path Dependent”

| Aspect                      | Sous $\mathbb P$ (real-world)                                                                                                                                  | Sous $\mathbb Q$ (pricing)                                                                                                                       |
| --------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------ |
| **But**                     | Prévoir/‐tester la volatilité réalisée, calculer VaR, stress, back-tests                                                                                       | Produire des prix cohérents avec toutes les options (SPX & VIX)                                                                                  |
| **Données utilisées**       | Historique SPX, RV 30 d, VIX spot (validation)                                                                                                                 | Surface IV SPX, futures & options VIX, strip OTM 30 j                                                                                            |
| **Paramètres qu’on ajuste** | $\beta$, $\theta$, $\lambda$ via régression EWMA → reflètent la **dynamique réelle**                                                                           | Les mêmes paramètres **+** « prix du risque de vol » $\eta$ s’ils ne suffisent pas → forcés par le marché                                        |
| **Drifts**                  | $dS_t = S_t\sigma_t\,dW_t^{\mathbb P}$ (drift retiré si on travaille en rendements) <br>$dR_{n,j,t}= \lambda_{n,j}\bigl(\sigma_t^n - R_{n,j,t}\bigr)dt +\dots$ | Même diffusion, mais on retranche les **primes de risque** :<br>$dR_{n,j,t}= \bigl[\lambda_{n,j}(\sigma_t^n-R_{n,j,t})-\eta_{n,j}\bigr]dt+\dots$ |
| **Quantités contraires**    | Corrélations réalisées SPX–VIX, moments empiriques                                                                                                             | Skew, term-structure, smile IV, niveau VIX                                                                                                       |
| **Méthode d’ajustement**    | Minimisation L² sur différences VIX réalisé / $\beta_0+\beta_1R_1+\beta_2\sqrt{R_2}$                                                                           | Calibration « Global L-S » ou L-BFGS-B sur volatilités implicites avec pénalité $\| \vartheta-\vartheta^{\mathbb P}\|^2$                         |
| **Interprétation**          | Paramètres = comportement moyen du marché (vol réalisée)                                                                                                       | Paramètres = **comportement sous risque-neutre** (intègre la prime de variance)                                                                  |
| **Utilisation**             | • Projections PnL, scenario analysis<br>• Mesure de risque (VaR, ES)<br>• Stress tests                                                                         | • Pricing SPX & VIX<br>• Greeks, hedging<br>• X-gamma / volatilité forward                                                                       |

**En pratique :**

1. **Étape P (estimation)** :

   * On ajuste $\beta,\theta,\lambda$ sur ≥ 10 ans de données, sans souci de prix d’option.
   * On obtient la dynamique “naturelle”.

2. **Étape Q (calibration)** :

   * On garde la même structure SDE mais on modifie les *drifts* (ou rajoute un simple décalage sur $\beta$) via un vecteur de primes $\eta$ pour coller aux options.
   * On minimise l’erreur sur IV SPX & VIX tout en pénalisant un trop grand écart par rapport aux valeurs $\mathbb P$.

3. **Mapping $\mathbb P \to \mathbb Q$** :

   * Formellement : $dX_t^{\mathbb Q}=dX_t^{\mathbb P}-\lambda_X\,dt$ où $\lambda_X$ est la **prime de risque** du facteur $X$.
   * Dans ce modèle on applique cela aux filtres $R_{n,j}$ (prime de risque de variance).

> **Résumé ultra-court :**
> • Sous $\mathbb P$ → paramètres reflètent l’histoire et servent au *risk management*.
> • Sous $\mathbb Q$ → on ajoute/ajuste les primes de risque pour reproduire **les prix d’option** ; le reste de la diffusion reste identique, donc le modèle reste Markovien dans $(S,R)$.


### Road-map « pricing » sous $\mathbb Q$ avec le **4-factor PDV** de Guyon & Lekeufack

*(calibré conjointement sur SPX & VIX)*

---

#### 1. Modèle risque-neutre

$$
\begin{aligned}
dS_t &= (r_t-q_t)S_t\,dt \;+\;S_t\,\sigma_t\,dW_t^{\mathbb Q},\\[4pt]
\sigma_t &= \beta_0^{\!Q}+\beta_1^{\!Q} R_{1,t}+\beta_2^{\!Q}\sqrt{R_{2,t}},\\[4pt]
dR_{1,j,t} &= \bigl[-\lambda_{1,j} R_{1,j,t}-\eta_{1,j}\bigr]dt+\lambda_{1,j}\sigma_t\,dW_t^{\mathbb Q},\\
dR_{2,j,t} &= \bigl[-\lambda_{2,j} R_{2,j,t}+\lambda_{2,j}\sigma_t^{2}-\eta_{2,j}\bigr]dt.
\end{aligned}
$$

* $j=0$ (mémoire courte) / $j=1$ (mémoire longue).
* $ \eta_{n,j}$ : **primes de risque de variance** à estimer (0 si l’on s’autorise un simple déplacement des $ \beta$).
* L’état $(S,R_{1,0},R_{1,1},R_{2,0},R_{2,1})$ est **Markovien** ; le modèle peut donc être pricé par MC ou PDE 5-D ([informaconnect.com][1]).

---

#### 2. Market-data minimale pour la calibration $\mathbb Q$

| Bloc                                                                   | Champ(s) précis                               | Rôle dans le coût     |
| ---------------------------------------------------------------------- | --------------------------------------------- | --------------------- |
| SPX **spot**, courbe **dividende** $q_t$, courbe **taux** $r_t$        | actualisation, forward                        | input fixe            |
| **Surface IV SPX** (1 sem → 3-5 ans, 5-10 strikes / maturité, bid/ask) | niveau + skew SPX                             | $\chi^2_{\text{SPX}}$ |
| **Strip OTM SPX 30 j** (largeur 50 %)                                  | nécessaire au calcul interne du VIX théorique | contrainte impl.      |
| **Futures VIX** (VX1…VX6, prix + échéance)                             | niveau term-structure                         | $\chi^2_{\text{VX}}$  |
| **Options VIX** (1–6 m, strikes 80–120 %)                              | skew VIX                                      | $\chi^2_{\text{VXO}}$ |

*(Le réalisé n’est plus requis : utile seulement pour back-test sous $\mathbb P$.)*

---

#### 3. Fonction objectif

$$
J(\vartheta)=
\underbrace{\sum_{i}w_i^{\text{SPX}}\bigl[\sigma^{\text{mod}}_{i}-\sigma^{\text{mkt}}_{i}\bigr]^2}_{\text{surface SPX}}
+\underbrace{\sum_{k}w_k^{\text{VX}}\bigl[F^{\text{mod}}_{k}-F^{\text{mkt}}_{k}\bigr]^2}_{\text{futures VIX}}
+\underbrace{\sum_{j}w_j^{\text{VXO}}\bigl[\sigma^{\text{mod}}_{j}-\sigma^{\text{mkt}}_{j}\bigr]^2}_{\text{options VIX}}
+\gamma\|\vartheta-\vartheta^{\mathbb P}\|_2^{2}
$$

* Pondération $w$= vega × 1/spread.
* Terme de rappel $\gamma$ pour éviter que $\beta,\lambda$ s’éloignent trop des valeurs historiques.

---

#### 4. Pipeline de pricing

| Étape                           | Action                                                                | Outil recommandé                                             |
| ------------------------------- | --------------------------------------------------------------------- | ------------------------------------------------------------ |
| **(a) Pré-processing**          | télécharger surfaces, futures, nettoyer bid/ask, interpoler IV        | Pandas / Parquet                                             |
| **(b) Warm-start $\mathbb P$**  | régression OLS/EWMA ⇒ $\beta,\lambda,\theta$                          | `empirical_study.ipynb` dans repo officiel ([github.com][2]) |
| **(c) Engine**                  | MC 5-facteurs (antithetic + Sobol) **OU** solveur PDE semi-Lagrangien | GPU / C++ (+ AAD pour gradients)                             |
| **(d) Calibration $\mathbb Q$** | L-BFGS-B ou CMA-ES sur $J(\vartheta)$                                 | `option_pricing_4fmpdv.ipynb` ([github.com][2])              |
| **(e) Pricing**                 | valuations SPX & VIX, greeks via pathwise/AAD                         | même moteur                                                  |
| **(f) Validation**              | vérifier erreurs relatives < 1 % SPX, < 3 % VIX; stress-test paths    | notebooks de contrôle                                        |

---

#### 5. Petits trucs pratiques

* **VIX théorique** :
  $\displaystyle VIX_t^{2}= \frac{2}{\Delta}\int_0^{\infty}\frac{(1\!-\!k/K)^+}{k^2}\,q_t(k)\,dk$
  où $q_t(k)$ est la densité neutre sur les strikes SPX pour maturité 30 j (numériquement : spline + trapèzes).
* **Greeks SPX** : dérivée pathwise classique ; pour VIX options utiliser différentiation automatique (AAD) car payoff ≠ lisse.
* **Stabilité** : impose $\beta_0^{\!Q}>0,\;|\beta_1^{\!Q}|<\beta_0^{\!Q},\;\beta_2^{\!Q}<1$ et $ \lambda_{n,0}>\lambda_{n,1}>0$ pour garantir $ \sigma_t>0$.

---

##### En résumé

1. **Collecte** : spot, courbes taux/div, surface SPX, strip 30 j, futures & options VIX.
2. **Calibration** $\mathbb Q$ via cost $J$ (MC ou PDE).
3. **Pricing** : même moteur donne prix & greeks SPX/VIX cohérents.

Tu disposes ainsi du jeu de données requis et de la marche à suivre pour déployer un pricer complet basé sur le 4-factor PDV markovien, exactement tel que présenté par Guyon & Lekeufack.

[1]: https://informaconnect.com/volatility-is-mostly-path-dependent/ "Volatility is (mostly) path-dependent"
[2]: https://github.com/Jordylek/VolatilityIsMostlyPathDependent "GitHub - Jordylek/VolatilityIsMostlyPathDependent: Code for the paper Volatility is (mostly) path-dependent"


### Calibration pratique des volatilités implicites

*(SPX + VIX) avec le modèle « Vol Mostly Path-Dependent »*

---

#### 1. Préparer la grille marché

* **Sélectionne** : strikes $K_i$, maturités $T_i$ (SPX) et $(k_j,\tau_j)$ (VIX).
* **Filtre** : garde uniquement les quotes liquides; stocke bid/ask et vega pour les pondérations.

---

#### 2. Boucle de calibration (paramètres $ \vartheta$)

| Étape                     | Détail métier                                                                                                                                                                                                                              | Implémentation rapide                                        |
| ------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | ------------------------------------------------------------ |
| **2-a. Pricing modèle**   | - Simule (MC) ou résout (PDE) les SDE sur $(S,R)$.<br>- Renvoie le **prix** théorique $C^{\text{mod}}_{i}$ / $P^{\text{mod}}_{i}$.                                                                                                         | MC antithétique + Sobol ; 200 k paths suffisent avec CV/AAD. |
| **2-b. Conversion en IV** | **Inversion Black-Scholes (SPX)** :<br>$\sigma^{\text{mod}}_{i} = \text{BrentRoot}\bigl(\sigma\mapsto C_{\text{BS}}(K_i,T_i,\sigma)-C^{\text{mod}}_{i}\bigr)$.<br>**Inversion Black-76 (VIX options)** pour $\sigma^{\text{mod,VIX}}_{j}$. | Brent ou Halley; tolérance $10^{-6}$.                        |
| **2-c. Fonction coût**    | $\displaystyle J(\vartheta)=\!\!\sum_{i}w_i(\sigma^{\text{mod}}_i-\sigma^{\text{mkt}}_i)^2+\!\!\sum_{j}\tilde w_j(\sigma^{\text{mod,VIX}}_j-\sigma^{\text{mkt,VIX}}_j)^2+\gamma\|\vartheta-\vartheta^{\mathbb P}\|^2$                      | $w=\text{vega}/\text{spread}$.                               |
| **2-d. Optimisation**     | L-BFGS-B / Adam / CMA-ES sous contraintes ($\beta_0>0$, etc.).<br>Gradients : **pathwise + AAD** (MC) ou adjoint PDE.                                                                                                                      | 20–40 itérations suffisent.                                  |

---

#### 3. Notes clés

1. **VIX théorique** pour futures/options

   $$
   VIX_t^2=\frac{2}{\Delta}\!\int\limits_{K_\text{min}}^{K_\text{max}}\!\frac{1-K/K_0}{K^2}q_t(K)\,dK,
   $$

   $q_t(K)$ : densité neutre extraite de la **même** surface SPX simulée — pas de paramètre supplémentaire.

2. **Régularisation**

   * Terme $\gamma$ maintient la proximité avec les paramètres $\mathbb P$ (historique).
   * Lissage éventuel sur la surface d’erreurs pour éviter le sur-ajustement des ailes sombres.

3. **Accélérations**

   * Pré-calcule les exponentielles $e^{-\lambda_{n,j}\Delta t}$.
   * Stocke les trajectoires des filtres $R$ pour réutiliser entre itérations (re-poids de chemins).

4. **Critères d’arrêt**

   * Erreur RMS < 1 % sur SPX et < 3 % sur VIX ou amélioration $<10^{-4}$ sur $J$.
   * Vérifie la stabilité : $ \sigma_t>0$ sur 3 σ des états simulés.

---

#### 4. Résultat

Une fois $\vartheta^\star$ obtenu, **le même moteur** fournit :

* prix et greeks SPX (calls/puts, cliquets…),
* prix et greeks VIX (futures/options),
* scénarios joint SPX–VIX pour le risk-management.

---

**TL;DR :**

1. Prix modèle → 2. Inversion BS/76 → 3. Erreur quadratique pondérée → 4. Optimisation avec gradients AAD.
   Cette boucle aligne les vols implicites du modèle sur celles du marché de façon robuste et rapide.


### Pipeline de calibration → pricing

*(modèle 4-facteurs PDV de Guyon-Lekeufack, “Volatility is (Mostly) Path-Dependent”)*

---

#### 1. Collecte & pré-processing

| Bloc                                        | Données                 | But                  |
| ------------------------------------------- | ----------------------- | -------------------- |
| SPX spot + dividendes + courbe OIS          | définir le forward      | actualisation        |
| Surface IV SPX (≈1 sem-5 ans, 5–10 strikes) | cible smile             | pondération par vega |
| Strip OTM 30 j (≈5 %-50 %)                  | reconstruire le VIX     | entrée formule VIX   |
| Futures & options VIX (1-6 m)               | niveau + skew VIX       | joint calibration    |
| Historique SPX close (≥10 ans)              | filtre EWMA $R_{n,j,t}$ | estime $P$-mesure    |

Nettoyer bid/ask, supprimer quotes illiquides, stocker en Parquet.

---

#### 2. **Fit historique ($\mathbb P$)** — `empirical_study.ipynb`

1. Calcule les retours $r_t=\log S_t-\log S_{t-\Delta t}$.
2. Met à jour les filtres exponentiels :

   $$
   \begin{aligned}
   R_{1,j,t+\Delta t}&=e^{-\lambda_{1,j}\Delta t}R_{1,j,t}+\lambda_{1,j}r_{t+\Delta t},\\
   R_{2,j,t+\Delta t}&=e^{-\lambda_{2,j}\Delta t}R_{2,j,t}+\lambda_{2,j}r_{t+\Delta t}^2.
   \end{aligned}
   $$
3. Régression OLS pénalisée
   $\sigma_{30\text{d}}^{\text{hist}}=\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$
   ⇒  $\widehat{\beta},\widehat{\lambda},\widehat{\theta}$.

> Notebook fourni par les auteurs : `empirical_study.ipynb` ([github.com][1])

---

#### 3. **Moteur de pricing (Markovien 5-D)**

* États : $X_t=(S_t,R_{1,0},R_{1,1},R_{2,0},R_{2,1})$.
* Simulation MC Sobol + antithétique (GPU) **ou** PDE 5-D semi-Lagrangien.
* Grecs : pathwise /AAD.
* Contraintes : $\beta_0>0,\;|\beta_2|<1,\;\lambda_{n,0}>\lambda_{n,1}>0$.

---

#### 4. **Surrogat VIX** (accélération)

Le papier 2024 propose un **réseau de neurones** entraîné sur des trajectoires simulées pour approximer

$$
VIX_t=f_{\text{NN}}\bigl(R_{1,0},R_{1,1},R_{2,0},R_{2,1};\vartheta\bigr)
$$

⇒ plus de nested MC : calibration × 60 plus rapide. ([arxiv.org][2])

---

#### 5. **Calibration risque-neutre ($\mathbb Q$)**

Fonction coût

$$
J(\vartheta)=\underbrace{\sum w_i(\sigma_i^{\text{mod}}-\sigma_i^{\text{mkt}})^2}_{\text{surface SPX}}
+\underbrace{\sum \tilde w_k(F_k^{\text{mod}}-F_k^{\text{mkt}})^2}_{\text{futures VIX}}
+\underbrace{\sum \bar w_j(\sigma_j^{\text{mod,VIX}}-\sigma_j^{\text{mkt,VIX}})^2}_{\text{options VIX}}
+\gamma\|\vartheta-\widehat{\vartheta}^{\mathbb P}\|^2  
$$

* Pondération $w=$ vega / spread.
* Optimiseur : **Py-BOBYQA** (trust-region, dérivées non requises) — 30 min surface SPX, 8 min joint SPX/VIX sur GPU NVIDIA selon le papier. ([arxiv.org][2])
* À chaque itération :

  1. MC → prix $C^{\text{mod}}$ SPX, VIX (futures + options via NN).
  2. Inversion Black-Scholes/Black-76 pour obtenir $\sigma^{\text{mod}}$.

---

#### 6. **Validation**

* Erreur RMS < 1 % (SPX) et < 3 % (VIX).
* Vérifie que $\sigma_t>0$ sur 3 σ des états simulés.
* Back-test stabilité paramètres (voir section 6 du papier). ([arxiv.org][2])

---

#### 7. **Pricing opérationnel** — `option_pricing_4fmpdv.ipynb`

Le même moteur fournit ensuite :

* prix & grecs SPX (vanilles, exotiques path-indépendantes),
* prix & grecs VIX futures / options,
* scénarios SPX-VIX cohérents pour le hedging.
  Notebook exemple livré par les auteurs : `option_pricing_4fmpdv.ipynb`. ([github.com][1])

---

##### Résumé ultra-compact

1. **Fit historique** EWMA → $\widehat{\beta},\widehat{\lambda},\widehat{\theta}$.
2. **Construis** moteur MC/PDE + NN VIX.
3. **Calibre $\mathbb Q$** sur surface SPX + VIX (Py-BOBYQA).
4. **Valide**, puis **price** n’importe quel dérivé SPX ou VIX.

[1]: https://github.com/Jordylek/VolatilityIsMostlyPathDependent "GitHub - Jordylek/VolatilityIsMostlyPathDependent: Code for the paper Volatility is (mostly) path-dependent"
[2]: https://arxiv.org/html/2406.02319v1 "Pricing and calibration in the 4-factor path-dependent volatility model"

LAAAAA


### Extraits clés : comment le notebook met à jour les « grands » $R_{n,j}$

```python
# --- initialisation des filtres à partir des prix historiques -------------
def initialize_R(lam, past_prices=None, transform=identity):
    returns   = 1 - past_prices.shift(1) / past_prices          # r_t
    returns   = torch.tensor(returns.values)[-max_delta:].flip(0)
    timestamps = torch.arange(returns.shape[0]) * dt
    weights    = lam.unsqueeze(1) * torch.exp(-lam.unsqueeze(1) * timestamps)
    x = transform(returns)                                      # r_t   ou   r_t²
    return torch.sum(x * weights, dim=1)                        # R_{n,j,0}
/*…*/                                                              :contentReference[oaicite:0]{index=0}
```

```python
# --- mise à jour à chaque pas de temps -----------------------------------
for t in range(n_timesteps):
    R1 = (1 - θ1) * r1[0] + θ1 * r1[1]           # R_{1,t}
    R2 = (1 - θ2) * r2[0] + θ2 * r2[1]           # R_{2,t}
    vol = β0 + β1 R1 + β2 √R2                    # σ_t

    brownian_increment = √dt * torch.randn(N)
    increment = vol * brownian_increment         # σ_t dW_t  (≈ r_t)

    for j in (0,1):                              # j = court / long
        r1[j] = exp(-λ1[j] dt) * (r1[j] + λ1[j] * increment)
        r2[j] = exp(-λ2[j] dt) * (r2[j] + λ2[j] * vol**2 * dt)
    # S_t mis à jour ensuite (Euler–Maruyama log-normal)
/*…*/                                                              :contentReference[oaicite:1]{index=1}
```

---

### Ce que fait exactement le code

| Terme                                            | Formule SDE du papier                                                     | Discrétisation utilisée |
| ------------------------------------------------ | ------------------------------------------------------------------------- | ----------------------- |
| $dR_{1,j,t}=λ_{1,j}(\sigma_t\,dW_t-R_{1,j,t})dt$ | $ r1[j] \leftarrow e^{-λ_{1,j}\,Δt}\bigl(r1[j]+λ_{1,j}\,σ_t dW_t\bigr)$   |                         |
| $dR_{2,j,t}=λ_{2,j}(\sigma_t^{2}-R_{2,j,t})dt$   | $ r2[j] \leftarrow e^{-λ_{2,j}\,Δt}\bigl(r2[j]+λ_{2,j}\,σ_t^{2} Δt\bigr)$ |                         |

* **Choix « exact discret »** : l’exponentielle $e^{-λΔt}$ correspond à la solution fermée de l’O.U. “forgetting factor”, d’où une mise à jour stable même pour de grands pas.
* **Incrément aléatoire** : `increment = σ_t dW_t` est directement injecté dans la formule de $R_{1,j}$.
* **Agrégation** : $R_{n,t}=(1-θ_n)R_{n,0,t}+θ_nR_{n,1,t}$ est recalculé avant chaque pas pour obtenir la vol instantanée $\sigma_t$.
* **Initialisation** : `initialize_R` reconstruit $R_{n,j,0}$ comme convolution exponentielle des $r_t$ passés ; la même logique que la SDE mais appliquée hors-ligne sur les données historiques.

En résumé, le notebook suit fidèlement la dynamique du papier :

* OU exponentiel pour les deux échelles de mémoire,
* schéma exact en temps discret via le facteur $e^{-λΔt}$,
* mise à jour conjointe avec le même incrément Brownien que celui du sous-jacent, garantissant la **Markovianité** et la cohérence des corrélations SPX-vol.


### Comment sont construits les trajectoires (\*paths\*) des filtres $R_{n,j}$ dans le notebook ?

> **Idée de base** : chaque $R_{n,j}$ est une *moyenne exponentielle* d’un signal (le retour $r_t$ ou son carré), c’est-à-dire un **processus d’Ornstein-Uhlenbeck**.
> La solution exacte entre $t$ et $t+\Delta t$ sert directement de schéma numérique ; on n’a donc **aucune erreur d’Euler** sur la partie « mémoire ».

---

## 1. Initialisation hors-ligne

1. **On lit un historique de prix** $S_{\text{hist}}$ (souvent 1–2 ans suffisent).
2. On calcule les *retours* $r_t=\log S_t-\log S_{t-\Delta t}$.
3. Pour chaque échelle $\lambda_{n,j}$ on applique la *convolution exponentielle* :

   $$
   R_{n,j,0}
     =\lambda_{n,j}\sum_{k=0}^{\infty}e^{-\lambda_{n,j}k\Delta t}\,
       \bigl[r_{t-k}\bigr]^{\,n}.
   $$

   Dans le code :

   ```python
   weights = lam * torch.exp(-lam * timestamps)   # e^{-λkΔt}
   R0      = (weights * transform(returns)).sum() # n=1 ou n=2
   ```

   👉 On obtient **une valeur de départ** par path pour $R_{1,0},R_{1,1},R_{2,0},R_{2,1}$.

---

## 2. Boucle de simulation $(t \to t+\Delta t)$

À chaque pas, pour chaque **chemin Monte-Carlo** :

| Étape                           | Formule utilisée                                                                                       | Commentaire                                                                 |
| ------------------------------- | ------------------------------------------------------------------------------------------------------ | --------------------------------------------------------------------------- |
| **a. Fusion des échelles**      | $R_{1,t}=(1-\theta_1)R_{1,0,t}+\theta_1R_{1,1,t}$<br>$R_{2,t}=(1-\theta_2)R_{2,0,t}+\theta_2R_{2,1,t}$ | agrège courte + longue mémoire                                              |
| **b. Vol instantanée**          | $\sigma_t=\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$                                                | dépend lin. de $R_1$ et $\sqrt{R_2}$                                        |
| **c. Incrément brownien**       | $\Delta W\sim\mathcal N(0,\Delta t)$;  $\text{inc}=\sigma_t\,\Delta W$                                 | sert **à la fois** au prix et à $R_{1,j}$                                   |
| **d. Mise à jour de $S$**       | $S_{t+\Delta t}=S_t\exp\bigl((r-q-\tfrac12\sigma_t^2)\Delta t+\sigma_t\Delta W\bigr)$                  | Euler-Maruyama en log                                                       |
| **e. Mise à jour de $R_{1,j}$** | $R_{1,j,t+\Delta t}=e^{-\lambda_{1,j}\Delta t}R_{1,j,t}+\lambda_{1,j}\,\text{inc}$                     | solution **exacte** de l’OU (la partie stochastique est $\sigma_t\Delta W$) |
| **f. Mise à jour de $R_{2,j}$** | $R_{2,j,t+\Delta t}=e^{-\lambda_{2,j}\Delta t}R_{2,j,t}+\lambda_{2,j}\,\sigma_t^{2}\Delta t$           | ici, l’intégrale est **déterministe**                                       |

👉 Les quatre filtres évoluent **sans approximation d’Euler** grâce au facteur $e^{-\lambda\Delta t}$.

---

## 3. Pourquoi c’est Markovien ?

L’état courant

$$
X_t=\bigl(S_t,\,R_{1,0,t},R_{1,1,t},R_{2,0,t},R_{2,1,t}\bigr)
$$

contient **toute l’information nécessaire** pour générer l’étape suivante ; on ne regarde jamais plus loin dans le passé – c’est exactement la propriété Markov.

---

## 4. Pseudo-code ultra-court

```python
for each path:
    # --- step 0 : init --------------------------
    S, R10, R11, R20, R21 = init_from_history(...)
    for n in range(N_steps):
        R1 = (1-θ1)*R10 + θ1*R11
        R2 = (1-θ2)*R20 + θ2*R21
        sigma = β0 + β1*R1 + β2*sqrt(R2)

        dW = sqrt(dt) * randn()
        incr = sigma * dW

        # spot
        S *= exp((r-q-0.5*sigma**2)*dt + incr)

        # filters
        R10 = exp(-λ10*dt)*(R10 + λ10*incr)
        R11 = exp(-λ11*dt)*(R11 + λ11*incr)
        R20 = exp(-λ20*dt)*(R20 + λ20*sigma**2*dt)
        R21 = exp(-λ21*dt)*(R21 + λ21*sigma**2*dt)
```

---

### À retenir

* Les $R_{n,j}$ suivent un **OU exponentiel**, discrétisé exactement (multiplicateur $e^{-\lambda\Delta t}$).
* Le même incrément Brownien qui déplace $S$ sert à mettre à jour $R_{1,j}$, assurant la bonne corrélation **prix ↔ vol**.
* Grâce à cette construction, le modèle reste Markovien en dimension 5, ce qui simplifie Monte-Carlo, PDE et différentiation automatique.


### Discrétisation explicite des filtres $R_{n,j,t}$

*(utilisée dans le notebook des auteurs – schéma « exact OU » stable pour tout $\Delta t$)*

| Filtre    | SDE continue (papier)                                               | Mise à jour discrète exacte $(t\!\to\!t+\Delta t)$                                                                 |
| --------- | ------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------ |
| $R_{1,j}$ | $dR_{1,j,t}= \lambda_{1,j}\bigl(\sigma_t\,dW_t-R_{1,j,t}\,dt\bigr)$ | $\boxed{R_{1,j,t+\Delta t}=e^{-\lambda_{1,j}\Delta t}\Bigl(R_{1,j,t}+\lambda_{1,j}\,\sigma_t\,\Delta W_t\Bigr)}$   |
| $R_{2,j}$ | $dR_{2,j,t}= \lambda_{2,j}\bigl(\sigma_t^{2}-R_{2,j,t}\bigr)dt$     | $\boxed{R_{2,j,t+\Delta t}=e^{-\lambda_{2,j}\Delta t}\Bigl(R_{2,j,t}+\lambda_{2,j}\,\sigma_t^{2}\,\Delta t\Bigr)}$ |

**Variables :**

* $n\in\{1,2\}$ (retour ou carré du retour)
* $j=0$ (mémoire courte), $j=1$ (mémoire longue)
* $\Delta W_t \sim\mathcal N(0,\Delta t)$ est **le même incrément Brownien** que celui utilisé pour simuler le sous-jacent $S_t$.
* $\sigma_t=\beta_0+\beta_1 R_{1,t}+\beta_2\sqrt{R_{2,t}}$.

---

#### Pourquoi « exact » ?

La solution de l’OU linéaire

$$
dX_t=-\lambda X_t\,dt + \lambda\,\xi_t
$$

se résout sur $[t,t+\Delta t]$ par
$X_{t+\Delta t}=e^{-\lambda\Delta t}\bigl(X_t+\lambda\int_t^{t+\Delta t}\!e^{\lambda(u-t)}\xi_u\,du\bigr)$.
En remplaçant $\xi_u$ par $\sigma_t\,dW_t$ (pour $n=1$) ou $\sigma_t^{2}\,dt$ (pour $n=2$) **et** en supposant $\sigma_t$ quasi-constant sur le pas, on obtient exactement les deux boîtes ci-dessus : pas d’erreur d’Euler, stabilité même pour grands $\lambda\Delta t$.

---

#### Implémentation (pseudo-Python)

```python
# incrément Brownien commun
dW  = sqrt(dt) * torch.randn(batch)      # ΔW_t
inc = sigma * dW                         # σ_t ΔW_t

# R_{1,j}
R1j = torch.exp(-lam1j*dt) * (R1j + lam1j * inc)

# R_{2,j}
R2j = torch.exp(-lam2j*dt) * (R2j + lam2j * sigma**2 * dt)
```

Ces deux lignes **suffisent** pour propager chaque filtre à chaque pas de temps dans votre Monte-Carlo ou PDE semi-Lagrangien.


### Oui : il faut **amorcer** les filtres

Pour simuler avec la discrétisation « exact OU », il faut une valeur de départ

$$
R_{n,j,0}\;=\;R_{n,j}(t_0)
$$

par *chemin* Monte-Carlo.
Le papier (et les notebooks) proposent une méthode **déterministe** : calculer cette valeur en appliquant la même moyenne exponentielle… sur les retours déjà observés jusqu’à $t_0$.

---

#### 1. Formule d’amorçage

$$
\boxed{
R_{n,j,0}\;=\;\lambda_{n,j}\!\!\sum_{k=1}^{K_{\max}}
e^{-\lambda_{n,j}k\Delta t}\;\bigl[r_{t_0-k}\bigr]^{\,n}}
$$

* $r_{t}=\log S_t-\log S_{t-\Delta t}$ (ou le carré pour $n=2$).
* $\Delta t$ : pas du jeu de données (daily, hourly…).
* **Somme finie** : on tronque à $K_{\max}$ observations, car le poids exponentiel décroit vite ; choisir

  $$
  K_{\max}\simeq \left\lceil\frac{7}{\lambda_{n,j}\Delta t}\right\rceil
  $$

  capture > 99 % de la masse.

---

#### 2. Implémentation (pseudo-Python)

```python
def init_filter(lam, returns, power):
    """lam : scalaire λ_{n,j}; returns : tensor [T]; power = 1 ou 2"""
    returns = returns[-K_max:].flip(0)              # plus récents en premier
    k       = torch.arange(len(returns)) * dt
    w       = lam * torch.exp(-lam * k)             # poids exponentiels
    return (w * returns.pow(power)).sum()
```

* pour $R_{1,0}$ et $R_{1,1}$ : `power=1`
* pour $R_{2,0}$ et $R_{2,1}$ : `power=2`

---

#### 3. Que faire si l’historique est court ?

1. **Stationnaire** – utiliser l’espérance sous la loi stationnaire de l’OU :

   $$
   \mathbb E[R_{1,j}] = 0,\qquad
   \mathbb E[R_{2,j}] = \mathbb E[\sigma^2] \approx (\beta_0+\beta_2^2) .
   $$

   \=> fixe un unique point de départ identique pour tous les chemins.
2. **Calibration conjointe** – considérer $R_{n,j,0}$ comme **paramètres latents** et les laisser s’ajuster pendant la calibration ; plus cher, rarement nécessaire.

---

#### 4. Pipeline complet résumé

| Étape | Action                                                                                                              |
| ----- | ------------------------------------------------------------------------------------------------------------------- |
| 1     | Récupère $N_{\text{hist}}$ retours $\{r_{t_0-k}\}_{k=1}^{K_{\max}}$                                                 |
| 2     | Calcule $R_{1,0,0},R_{1,1,0},R_{2,0,0},R_{2,1,0}$ via la formule ci-dessus                                          |
| 3     | Lance la simulation : à chaque pas applique les mises à jour exactes  $e^{-\lambda\Delta t}( \cdots )$ déjà données |
| 4     | Utilise les paths pour le pricing et/ou la calibration des paramètres $\vartheta$                                   |

Avec ces valeurs d’amorce, **chaque trajectoire** $\bigl(S_t,R_{1,0,t},R_{1,1,t},R_{2,0,t},R_{2,1,t}\bigr)$ est bien définie dès $t_0$, et tu peux dérouler la discrétisation sans aucune ambiguïté.


Exact : l’itération

$$
R_{1,j,t+\Delta t}=e^{-\lambda_{1,j}\Delta t}R_{1,j,t}+\lambda_{1,j}\,r_{t+\Delta t}
$$

**ne suffit pas** ; il faut impérativement **fixer la condition initiale** $R_{1,j,0}$ (et de même $R_{2,j,0}$) parce que :

* le modèle est Markovien dans $\bigl(S_t,R_{1,0,t},R_{1,1,t},R_{2,0,t},R_{2,1,t}\bigr)$ ; sans $R_{n,j,0}$ l’état n’est pas défini ;
* la volatilité instantanée $\sigma_t=\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$ doit être cohérente dès $t=0$ avec la surface d’options / le VIX.

### Comment choisir $R_{n,j,0}$ en pratique ?

| Méthode                              | Formule ou idée                                                                                                                                                      | Quand l’utiliser                                                                           |
| ------------------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------ |
| **EWMA tronquée (la plus courante)** | $\displaystyle R_{n,j,0}=\lambda_{n,j}\sum_{k=1}^{K_{\max}}e^{-\lambda_{n,j}k\Delta t}\,[r_{-k}]^{\,n}$ avec $K_{\max}\simeq \lceil 7/(\lambda_{n,j}\Delta t)\rceil$ | on dispose d’un historique de prix (quelques années daily suffisent)                       |
| **Distribution stationnaire**        | $R_{1,j,0}=0,\quad R_{2,j,0}=\mathbb E[\sigma^2]\approx(\beta_0+\beta_2^2)$                                                                                          | historique très court ou simulation de scénarios « moyens »                                |
| **Variable latente calibrée**        | traiter chaque $R_{n,j,0}$ comme paramètre et l’ajuster contre la surface IV (rajoute 4 dim.)                                                                        | pricing pur *sans* données historiques mais avec besoin d’adéquation très fine au VIX spot |

> **EWMA tronquée** est celle utilisée dans les notebooks : on remonte dans les retours jusqu’à ce que le facteur $e^{-\lambda k\Delta t}$ devienne négligeable (<10-6). Pour des $\lambda$ usuels (0,5 → 10 ans$^{-1}$) et un pas quotidien, 500 à 2 000 retours suffisent.

### Pourquoi l’amorce est vite « oubliée » ?

Après $m$ pas, la contribution de $R_{n,j,0}$ est multipliée par $e^{-\lambda_{n,j}m\Delta t}$.
Même si on partait de $R_{n,j,0}=0$, l’erreur sur $R_{n,j,t}$ tombe sous 1 % dès
$m > 5/(\lambda_{n,j}\Delta t)$.
Mais pour **pricing au temps 0**, on veut tout de suite la bonne volatilité ; d’où le calcul précis.

### Résumé opérationnel

1. **Choisis** un horizon $K_{\max}$ (ex. 3 ans de daily → ≈ 750 retours).
2. **Calcule** $R_{1,0,0},R_{1,1,0},R_{2,0,0},R_{2,1,0}$ avec la formule EWMA tronquée.
3. **Lance** la discrétisation « exacte OU » à chaque pas :

   $$
   \begin{aligned}
   R_{1,j,t+\Delta t}&=e^{-\lambda_{1,j}\Delta t}\bigl(R_{1,j,t}+\lambda_{1,j}\sigma_t\Delta W_t\bigr),\\
   R_{2,j,t+\Delta t}&=e^{-\lambda_{2,j}\Delta t}\bigl(R_{2,j,t}+\lambda_{2,j}\sigma_t^{2}\Delta t\bigr).
   \end{aligned}
   $$
4. **Calibre** ensuite les paramètres $\vartheta$ sous $\mathbb Q$ (surface SPX + VIX).

Ainsi le path des $R_{n,j}$ est entièrement défini : condition initiale cohérente, puis évolution stable et sans biais grâce au schéma exponentiel exact.


### Où apparaissent σᵗ et σᵗ² dans la mise à jour

*(et pourquoi on n’emploie pas directement le retour rₜ₊Δₜ pour R₂)*

| Symbole   | Signification au pas **t**                  | Comment on le calcule dans le code       |
| --------- | ------------------------------------------- | ---------------------------------------- |
| **σₜ**    | volatilité instantanée donnée par le modèle | `sigma = β0 + β1*R1 + β2*torch.sqrt(R2)` |
| **σₜ²**   | variance instantanée                        | simplement `sigma**2` (le carré)         |
| **rₜ₊Δₜ** | retour simulé sur le pas                    | `incr = sigma * dW   # ≡ rₜ₊Δₜ`          |

---

### Mise à jour « exact OU » rappelée

$$
\boxed{
\begin{aligned}
R_{1,j,t+\Delta t} &= e^{-λ_{1,j}Δt}\left(R_{1,j,t}+λ_{1,j}\,\underbrace{σ_tΔW_t}_{r_{t+Δt}}\right) \\[4pt]
R_{2,j,t+\Delta t} &= e^{-λ_{2,j}Δt}\left(R_{2,j,t}+λ_{2,j}\,\underbrace{σ_t^{2}}_{\text{variance inst.}}Δt\right)
\end{aligned}}
$$

* **R₁** : on utilise **le retour réalisé** `r = σ ΔW` (déjà dans la formule).
* **R₂** : on utilise **le carré de la volatilité, pas du retour**.

---

### Pourquoi ne pas remplacer σₜ²Δt par rₜ₊Δₜ² ?

1. **Sens de R₂**
   $R_{2,j}$ est une *moyenne exponentielle de la variance* (processus à dérive pure) :

   $$
   dR_{2,j,t}=λ_{2,j}\bigl(σ_t^{2}-R_{2,j,t}\bigr)dt .
   $$

   Le terme d’intégration est donc l’**espérance conditionnelle** de $r^2/Δt$, pas le carré réalisé lui-même.

2. **Bruit numérique**
   Remplacer $σ_t^{2}Δt$ par $r^2$ apporterait un bruit $\mathcal O(Δt)$ qui, pour un pas journalier, est très supérieur à la précision voulue lors d’une calibration (il dégrade la convergence de l’optimiseur).

3. **Impartialité vs. variance**

   * $E[r^2\mid\sigma_t]=\sigma_t^2 Δt$ → utiliser $r^2$ est **sans biais**.
   * Mais la *variance* supplémentaire est $2\sigma_t^4Δt^2$ : négligeable en continu, significative en discret.
     Les auteurs préfèrent donc la version déterministe pour stabiliser l’algorithme.

> ⚠️ Rien ne t’empêche, pour un back-test de volatilité réalisée, d’essayer
> $R_{2,j,t+\Delta t}=e^{-λ_{2,j}Δt}\bigl(R_{2,j,t}+λ_{2,j}\,r_{t+Δt}^{2}\bigr)$.
> Mais pour la **calibration/pricing** c’est plus bruyant et ralentit l’optimisation.

---

### Procédure complète (rappel)

1. **Amorçage**
   $\displaystyle R_{n,j,0}=λ_{n,j}\sum_{k=1}^{K_{\max}}e^{-λ_{n,j}kΔt}\,[r_{-k}]^{n}$
   avec historique suffisant ou, à défaut, valeur stationnaire simple.

2. **Boucle de simulation MC**

   ```python
   dW   = sqrt(dt) * torch.randn(batch)
   r    = sigma * dW                 # retour simulé
   S   *= exp((r-q-0.5*sigma**2)*dt + r)

   R1j = exp(-lam1j*dt) * (R1j + lam1j * r)           # utilise le retour
   R2j = exp(-lam2j*dt) * (R2j + lam2j * sigma**2*dt) # utilise σ²
   ```

3. **Calibration**
   – boucle coût ↔ optimisation sur la surface SPX + VIX, en gardant ce schéma numérique stable.

En bref : **σₜ** vient directement de la formule du modèle, on ré-emploie **r = σ ΔW** dans R₁, et on conserve **σ²Δt** (plutôt que $r^2$) pour R₂ afin de respecter la dynamique théorique et garantir une calibration fiable.


**Réponse courte :**
Oui, à chaque itération de calibration il faut régénérer les trajectoires $S_t,R_{n,j,t}$ parce que la volatilité
$\sigma_t(\vartheta)=\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$ change avec les nouveaux paramètres $\vartheta$.
On le fait toutefois **sans re-tirer les bruits** : on réutilise les *mêmes* incréments brownien $\Delta W$ (CRN), ce qui garde la variance basse et rend le recalcul quasi-linéaire en temps.

---

## 1. Cycle « une itération de l’optimiseur »

```
(1)  θ  ← nouveaux paramètres   (β, λ, θ, etc.)
(2)  R0 ← initialise les 4 filtres avec ces λ       #  micro-coût
(3)  S , R ← propage paths avec ΔW pré-générés      #  GPU, 1–2 ms
(4)  Prix  → IV → coût J(θ)                         #  idem
(5)  Optimiseur met θ à jour → retour (1)
```

### Pourquoi il faut rejouer tout le path

* $\sigma_t$ dépend **récursivement** de $R_t$ qui dépend de $\sigma_t$ : un simple re-pondérage des anciens paths n’est pas valide.
* Les ∆W identiques garantissent que les écarts de prix d’une itération à l’autre proviennent **uniquement** du changement de $\vartheta$ → gradients plus lisses.

---

## 2. Initialisation $R_{n,j,0}(\lambda)$ à chaque itération

Forme vectorielle (EWMA tronquée) :

$$
R_{n,j,0} =
\lambda_{n,j}\,
\bigl[w(\lambda_{n,j})\bigr]^{\!\top}\![r_{-k}]^{n},
\quad
w_k(\lambda)=e^{-\lambda k\Delta t}
$$

* **Pré-compute** une matrice $V_{k}= [r_{-k},\;r_{-k}^2]$ de taille $K_{\max}\times2$.
* Pour les deux λ courts et deux λ longs : un simple `torch.mv` → 4 produits scalaires; coût négligeable (< 10 µs).

---

## 3. Trucs pour ne pas exploser le temps de calibration

| Levier                          | Idée                                       | Gain constaté                    |
| ------------------------------- | ------------------------------------------ | -------------------------------- |
| **CRN** (Common Random Numbers) | même graine Brownien pour chaque itération | bruit divisé par ≈ 4             |
| **GPU**                         | vecteur 200 k paths × 5 facteurs           | 1 it ≈ 3 ms sur une RTX A5000    |
| **Adjoint/AAD**                 | grad pathwise en un backward unique        | x5 plus rapide qu’un finite-diff |
| **Surrogat VIX**                | réseau pré-entraîné pour éviter le *strip* | × 30 sur la partie VIX (papier)  |
| **Warm-start**                  | $\vartheta_{0}=\vartheta^{\mathbb P}$      | calibration < 30 it.             |

---

## 4. Quand on **n’a pas** d’historique suffisant

* **Option 1 :** mettre $R_{1,j,0}=0$ et $R_{2,j,0}=\beta_0^2$ puis
  *laisser la calibration déplacer $\beta$ pour coller au VIX spot* – marche car l’erreur initiale est vite oubliée ($<\!2$ semaines avec λ > 1/an).
* **Option 2 :** considérer $R_{n,j,0}$ comme 4 paramètres libres et les ajouter à $\vartheta$ (augmente la dimension, rarement nécessaire).

---

### TL;DR

* À **chaque itération** : mêmes ∆W, même historique $r_{-k}$ → on régénère $S,R$ vite et proprement.
* L’initialisation $R_{n,j,0}$ se recalcule en $\mathcal O}(K_{\max})$ grâce à un produit vectoriel.
* Avec GPU + CRN + AAD, la loop coût/pricing reste < 100 ms, donc la calibration globale se fait en quelques dizaines de secondes.


**Non : ils ne “figent” pas les paramètres.**

| Où ?                                 | Ce qui est vraiment « choisi à la main »                                                                                                          | Ce qui est ensuite **optimisé automatiquement**                                                                                                                                                                                               |
| ------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Papier (section 4, tableau 2)**    | - Deux ordres de grandeur pour les vitesses de mémoire :<br>• “short” ≃ 4–6 an⁻¹ (≈ 2–3 mois de demi-vie)<br>• “long”  ≃ 0,3–0,6 an⁻¹ (≈ 1–2 ans) | - Sur ces valeurs repères, ils lancent une **régression OLS** sur VIX pour obtenir β₀, β₁, β₂ et retiennent le couple (λ, θ) qui minimise l’erreur.                                                                                           |
| **Notebook `empirical_study.ipynb`** | - Même petite grille (4 × 4 combinaisons).<br>- Pas d’autre réglage manuel.                                                                       | - Boucle OLS → calcul automatique du meilleur quadruplet (β, λ, θ). Ce quadruplet sert de **warm-start** ; il est ensuite raffiné par l’optimiseur (L-BFGS-B ou Py-BOBYQA) lors de la calibration pricing dans `option_pricing_4fmpdv.ipynb`. |

**En pratique :**

1. Les vitesses de décroissance ($\lambda_{n,j}$) et les pondérations $\theta_n$ sont *initialement* prises dans une petite grille “raisonnable” définie **à la main** (cohérente avec les horizons de marché).
2. Dès que la grille est testée :

   * **β₀, β₁, β₂** sont déterminés **par régression** (aucune valeur imposée).
   * Le quadruplet qui donne la plus faible MSE sur le spot VIX devient le **point de départ**.
3. Dans la phase de calibration $\mathbb Q$ (surface SPX + VIX), **tous** ces paramètres (β, λ, θ) sont laissés libres et optimisés automatiquement ; le “choix à la main” n’influence donc que la vitesse de convergence, pas le résultat final.


dans la phase 1 on initialise a la main les lambda et teta, on fait la regression on utilise l approximation biaisé de la dynamique de R avec la discretisation avec le return r, on aura ici juste besoin de calculer R1,j,0 (encore une fois que avec les return r) et dans la phase 2 on prend les valeurs de beta obtenu par OLS, on reprend les meme lambda et teta qu en phase 1 et cette fois on construit les R avec dynamique OU avec le sigma qui sera recalculer a partir des beta. et dans cette phase on optimise tout les parametre : les lambda, les teta et les betas.



### Où tu as **bon ✓**

* ✓ **Grille initiale (λ & θ) choisie à la main** puis testée rapidement.
* ✓ **Phase 1 : OLS** sur VIX qui fournit un premier triplet $(β_0,β_1,β_2)$ sans jamais simuler l’SDE.
* ✓ Les coefficients $(β,λ,θ)$ issus de la phase 1 servent de **warm-start** pour la calibration pricing (phase 2).
* ✓ **Phase 2 :** on construit les trajectoires avec la vraie dynamique OU, on recalcule $\sigma_t=β_0+β_1R_{1,t}+β_2\sqrt{R_{2,t}}$ à chaque pas, et on **optimise de nouveau** β, λ et θ pour coller à la surface SPX + VIX.

### Où c’est **à corriger ✗**

* ✗ **Phase 1 utilise toute la série** $R_{n,j,t}$ (pour chaque t) — pas seulement $R_{1,j,0}$.
* ✗ Il faut **deux familles** de filtres : $R_1$ (retours) **et** $R_2$ (retours au carré).
* ✗ L’EWMA basée sur les retours n’est **pas biaisée** ; elle est l’expression exacte du filtre historique sous les données observées.
* ✗ L’initialisation pour la MC en phase 2 demande **les quatre valeurs** $R_{1,0,0},R_{1,1,0},R_{2,0,0},R_{2,1,0}$ obtenues par la même EWMA, pas seulement $R_{1,j,0}$.
* ✗ Dans certains codes les auteurs **laissent θ fixes** (ex. θ = 0.2) et n’optimisent que β et λ ; d’autres variantes les laissent libres — à décider selon ta robustesse numérique.


Oui, c’est exactement ça :

* **Phase 1 (historique / OLS)**

  * On dispose seulement des rendements observés $r_t = \ln S_t - \ln S_{t-\Delta t}$.
  * Les filtres sont construits *directement* avec ces rendements :

    $$
    \begin{aligned}
    R_{1,j,t} &= e^{-\lambda_{1,j}\Delta t}\,R_{1,j,t-\Delta t} + \lambda_{1,j}\,r_t,\\
    R_{2,j,t} &= e^{-\lambda_{2,j}\Delta t}\,R_{2,j,t-\Delta t} + \lambda_{2,j}\,r_t^{2}.
    \end{aligned}
    $$
  * Aucun $\sigma_t$ n’intervient ; on n’a donc ni boucle circulaire ni approximation biaisée : c’est la définition empirique du filtre exponentiel.

* **Pourquoi c’est correct**

  * Sous pas quotidien, $r_t \approx \sigma_t\sqrt{\Delta t}$, et l’erreur se retrouve absorbée dans les coefficients $\beta$ estimés par l’OLS.
  * Les $R_{n,j,t}$ ainsi obtenus sont les mêmes que si l’on appliquait la convolution exponentielle fermée $\lambda_{n,j}\sum e^{-\lambda k\Delta t} r_{t-k}^{\,n}$.

* **Phase 2 (pricing / calibration $\mathbb Q$)**

  * On part des mêmes $\lambda,\theta$ et des $\beta$ issus de l’OLS.
  * Les trajectoires $R_{n,j,t}$ sont désormais mises à jour avec la **dynamique OU** utilisant $\sigma_t = \beta_0 + \beta_1 R_{1,t} + \beta_2 \sqrt{R_{2,t}}$.
  * Tous les paramètres $(\beta, \lambda, \theta)$ peuvent alors être raffinés par l’optimiseur pour coller aux options SPX + VIX.

En résumé : **Phase 1 → filtres via rendements (pas de $\sigma$) ; Phase 2 → filtres via schéma OU + $\sigma$**.


### EWMA = **Exponentially Weighted Moving Average**

Moyenne mobile pondérée de façon exponentielle : les observations les plus récentes comptent davantage que les anciennes.

#### Forme discrète (pas Δt)

$$
M_t \;=\; (1-\alpha)\,x_t \;+\; \alpha\,M_{t-1}, 
\quad 0<\alpha<1
$$

* $x_t$ : dernière observation (rendement, variance, etc.)
* $M_{t-1}$ : moyenne au pas précédent
* $\alpha = e^{-\lambda\,\Delta t}$ définit la **vitesse d’oubli**
  (→ demi-vie $h_{1/2} = \tfrac{\ln 2}{\lambda}$)

#### Forme continue équivalente

$$
dM_t = \lambda\,(x_t - M_t)\,dt
$$

Cette SDE d’Ornstein–Uhlenbeck est celle qu’on utilise pour les filtres $R_{n,j,t}$ du modèle :

$$
R_{1,j,t}= \lambda_{1,j}\!\int_{-\infty}^{t} e^{-\lambda_{1,j}(t-u)}\,r_u\,du,
\quad
R_{2,j,t}= \lambda_{2,j}\!\int_{-\infty}^{t} e^{-\lambda_{2,j}(t-u)}\,r_u^{2}\,du.
$$

#### Pourquoi on l’aime

* **Mémoire décroissante** : plus réactif qu’une moyenne simple.
* **Facile à mettre à jour** : une seule récursion, pas besoin de stocker tout l’historique.
* **Stable numériquement** : le facteur exponentiel garantit que les poids anciens deviennent rapidement négligeables.

Dans notre calibration, on applique simplement l’EWMA aux rendements ($r_t$ ou $r_t^2$) pour construire les filtres $R_{n,j,t}$ sans connaître $\sigma_t$ a priori.


### Départ : définitions continues du papier

$$
\boxed{
\begin{aligned}
R_{1,j}(t)&=\lambda_{1,j}\int_{-\infty}^{t}\!e^{-\lambda_{1,j}(t-u)}\frac{dS_u}{S_u},\\[2mm]
R_{2,j}(t)&=\lambda_{2,j}\int_{-\infty}^{t}\!e^{-\lambda_{2,j}(t-u)}\Bigl(\frac{dS_u}{S_u}\Bigr)^2.
\end{aligned}}
\tag{1}
$$

En posant $r_t=\dfrac{dS_t}{S_t}= \sigma_t\,dW_t$ et $dt$ le pas unitaire (jour dans les données), on obtient les SDE de type Ornstein–Uhlenbeck :

$$
\boxed{
\begin{aligned}
dR_{1,j,t}&=\lambda_{1,j}\bigl(\sigma_t\,dW_t-R_{1,j,t}\,dt\bigr),\\
dR_{2,j,t}&=\lambda_{2,j}\bigl(\sigma_t^{2}-R_{2,j,t}\bigr)dt.
\end{aligned}}
\tag{2}
$$

---

### Solution exacte de l’OU sur un pas $\Delta t$

Pour une équation $dX_t=-\lambda X_t\,dt+\lambda\,\xi_t$,
la solution sur $[t,t+\Delta t]$ est

$$
X_{t+\Delta t}=e^{-\lambda\Delta t}\Bigl(X_t+\lambda\!\!\int_{t}^{t+\Delta t}\!e^{\lambda(u-t)}\xi_u\,du\Bigr).
\tag{3}
$$

---

### Application à nos filtres

#### $R_{1,j}$, (corrigé en fonction du **retour réalisé**)

Avec $\xi_u=\sigma_t\,dW_u$ (on fige $\sigma$ sur le pas), l’intégrale est
$\displaystyle \sigma_t\!\int_{t}^{t+\Delta t}\!e^{-\lambda_{1,j}(t+\Delta t-u)}dW_u$.

En première ordre (comme dans le notebook) on remplace l’exponentielle par 1,
et on note $\Delta W_t=W_{t+\Delta t}-W_t$.
On obtient alors :

$$
\boxed{%
R_{1,j,t+\Delta t}=e^{-\lambda_{1,j}\Delta t}\Bigl(R_{1,j,t}
+\lambda_{1,j}\,\sigma_t\,\Delta W_t\Bigr)}.
\tag{4}
$$

En posant le **retour simulé** $r_{t+\Delta t}=\sigma_t\,\Delta W_t$ :

$$
\boxed{%
\;R_{1,j,t+\Delta t}=e^{-\lambda_{1,j}\Delta t}R_{1,j,t}
+\lambda_{1,j}\,r_{t+\Delta t}}\tag{4′}
$$

— c’est exactement la ligne utilisée dans `option_pricing_4fmpdv.ipynb`.

---

#### $R_{2,j}$, (corrigé en fonction de **$\sigma_t^2$**)

Ici $\xi_u=\sigma_t^{2}\,dt$ (constante sur le pas).
L’intégrale vaut $\sigma_t^{2}\int_{t}^{t+\Delta t}\!e^{-\lambda_{2,j}(t+\Delta t-u)}du = \sigma_t^{2}\,\dfrac{1-e^{-\lambda_{2,j}\Delta t}}{\lambda_{2,j}}$.

Multipliée par $\lambda_{2,j}$ et remise dans (3) :

$$
\boxed{%
R_{2,j,t+\Delta t}=e^{-\lambda_{2,j}\Delta t}\Bigl(R_{2,j,t}
+\lambda_{2,j}\,\sigma_t^{2}\,\Delta t\Bigr)}.
\tag{5}
$$

C’est la mise à jour exacte (choisie par les auteurs) et codée dans le notebook.

*(On pourrait remplacer $\sigma_t^{2}\Delta t$ par $r_{t+\Delta t}^{2}$ : sans biais mais plus bruité ; le papier/notebook retient la version ci-dessus pour la stabilité de la calibration).*

---

### Formules discrètes finales utilisées dans le repo

$$
\boxed{
\begin{aligned}
R_{1,j}^{n+1}&=e^{-\lambda_{1,j}\Delta t}\bigl(R_{1,j}^{n}+\lambda_{1,j}\,r^{n+1}\bigr),\\[2mm]
R_{2,j}^{n+1}&=e^{-\lambda_{2,j}\Delta t}\bigl(R_{2,j}^{n}+\lambda_{2,j}\,(\sigma^{n})^{2}\,\Delta t\bigr),
\end{aligned}}
\tag{6}
$$

avec
$r^{n+1}=\sigma^{n}\Delta W^{n}$ et
$\sigma^{n}=\beta_0+\beta_1 R_{1}^{n}+\beta_2\sqrt{R_{2}^{n}}$.

Ces deux équations uniques sont **exactement** celles que l’on retrouve dans les cellules « simulate paths » du notebook des auteurs ; elles découlent directement des définitions (1) et de la résolution d’(2).


### 1. Point de départ : la définition continue dans le papier

Pour chaque $n\in\{1,2\}$ et chaque vitesse $j\in\{0,1\}$

$$
\boxed{R_{n,j,t}\;=\;\int_{-\infty}^{t}\lambda_{n,j}\,
e^{-\lambda_{n,j}(t-u)}\Bigl(\tfrac{dS_u}{S_u}\Bigr)^{n}}\qquad(\text{éq. 4.10})\;:contentReference[oaicite:3]{index=3}
$$

---

### 2. Dynamique différentielle (le papier la donne déjà)

En appliquant Itô à l’intégrale exponentielle :

$$
\boxed{\begin{aligned}
dR_{1,j,t}&=\lambda_{1,j}\bigl(\sigma_t\,dW_t-R_{1,j,t}\,dt\bigr)\\
dR_{2,j,t}&=\lambda_{2,j}\bigl(\sigma_t^{2}-R_{2,j,t}\bigr)\,dt
\end{aligned}}\qquad(\text{bloc 4.11})\;:contentReference[oaicite:4]{index=4}
$$

où $\sigma_t=\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$.

---

### 3. Résolution exacte d’un OU sur un pas $\Delta t$

Pour un processus linéaire
$dX_t=-\lambda X_t\,dt+\lambda\,\xi_t$
la solution entre $t$ et $t+\Delta t$ est :

$$
X_{t+\Delta t}=e^{-\lambda\Delta t}\Bigl[X_t+\lambda\!\!\int_{t}^{t+\Delta t}\!e^{\lambda(u-t)}\xi_u\,du\Bigr].
$$

On l’applique séparément à $R_{1,j}$ et $R_{2,j}$.

---

#### 3.1 Filtre « retour » $R_{1,j}$

*On fige $\sigma$ sur le pas* ($\sigma_u\approx\sigma_t$) et on note $\Delta W_t=W_{t+\Delta t}-W_t$.
L’intégrale stochastique vaut alors $\sigma_t\Delta W_t$.
D’où :

$$
\boxed{R_{1,j,t+\Delta t}=e^{-\lambda_{1,j}\Delta t}\Bigl(R_{1,j,t}+\lambda_{1,j}\,\sigma_t\Delta W_t\Bigr)}.
$$

En remplaçant $\sigma_t\Delta W_t$ par le **retour simulé** $r_{t+\Delta t}$ :

$$
\boxed{R_{1,j,t+\Delta t}=e^{-\lambda_{1,j}\Delta t}R_{1,j,t}+\lambda_{1,j}\,r_{t+\Delta t}}.
$$

---

#### 3.2 Filtre « variance » $R_{2,j}$

Ici l’intégrande est déterministe ($\xi_u=\sigma_t^{2}\,dt$), d’où

$$
\boxed{R_{2,j,t+\Delta t}=e^{-\lambda_{2,j}\Delta t}\Bigl(R_{2,j,t}+\lambda_{2,j}\,\sigma_t^{2}\Delta t\Bigr)}.
$$

---

### 4. Formules discrètes uniques utilisées dans le code

$$
\boxed{\begin{aligned}
R_{1,j}^{\,\text{new}}&=e^{-\lambda_{1,j}\Delta t}\bigl(R_{1,j}^{\,\text{old}}+\lambda_{1,j}\,r^{\text{new}}\bigr),\\[2pt]
R_{2,j}^{\,\text{new}}&=e^{-\lambda_{2,j}\Delta t}\bigl(R_{2,j}^{\,\text{old}}+\lambda_{2,j}\,\sigma_{\text{old}}^{2}\,\Delta t\bigr),
\end{aligned}}
$$

avec
$r^{\text{new}}=\sigma_{\text{old}}\Delta W$,
$\sigma_{\text{old}}=\beta_0+\beta_1R_{1}^{\,\text{old}}+\beta_2\sqrt{R_{2}^{\,\text{old}}}$.

Ces deux récursions sont exactement celles implémentées dans le notebook (boucle **simulate paths**) et proviennent directement de la définition intégrale (4.10) suivie de la résolution exacte de l’Ornstein–Uhlenbeck.

---

#### Remarque : variante EWMA historique

Pour la phase OLS où l’on ne dispose que des rendements observés, on remplace
$1-e^{-\lambda\Delta t}$ par $\lambda\Delta t$ et $\sigma_t\Delta W_t$ par le retour réalisé $r_t$.
On obtient alors la formule simple

$R_{1,j,t}=e^{-\lambda_{1,j}\Delta t}R_{1,j,t-\Delta t}+\lambda_{1,j}\,r_t$

utilisée uniquement pour **construire les variables explicatives** avant calibration.

---

**En conclusion :** les recurrences discrètes ci-dessus sont la transposition exacte, sans approximation d’Euler, des définitions continues du papier ; elles constituent la seule mise à jour employée dans la simulation Monte-Carlo du modèle à 4 facteurs.


### Récurrences “phase 1” : on ne dispose **que des rendements**

(aucune σ simulée, pas journalier Δt ≈ 1/252 an)

---

#### Rendement d’entrée

$$
r_t=\ln S_t-\ln S_{t-\Delta t}\qquad(\text{observé})
$$

---

#### Filtres élémentaires

| Facteur                | Mémoire rapide $j=0$                                                            | Mémoire lente $j=1$                                                             |
| ---------------------- | ------------------------------------------------------------------------------- | ------------------------------------------------------------------------------- |
| **Retour** $R_{1,j}$   | $R_{1,0,t}=e^{-\lambda_{1,0}\Delta t}R_{1,0,t-\Delta t}+\lambda_{1,0}\,r_t$     | $R_{1,1,t}=e^{-\lambda_{1,1}\Delta t}R_{1,1,t-\Delta t}+\lambda_{1,1}\,r_t$     |
| **Variance** $R_{2,j}$ | $R_{2,0,t}=e^{-\lambda_{2,0}\Delta t}R_{2,0,t-\Delta t}+\lambda_{2,0}\,r_t^{2}$ | $R_{2,1,t}=e^{-\lambda_{2,1}\Delta t}R_{2,1,t-\Delta t}+\lambda_{2,1}\,r_t^{2}$ |

---

#### Agrégation par échelle

$$
\boxed{\;
\begin{aligned}
R_{1,t}&=(1-\theta_1)R_{1,0,t}+\theta_1R_{1,1,t},\\
R_{2,t}&=(1-\theta_2)R_{2,0,t}+\theta_2R_{2,1,t}.
\end{aligned}}
$$

---

#### Initialisation (une seule fois)

$$
R_{n,j,0}
=\lambda_{n,j}\sum_{k=1}^{K_{\max}}
e^{-\lambda_{n,j}k\Delta t}\,[r_{-k}]^{\,n},
\quad
K_{\max}\;\text{tel que}\;e^{-\lambda_{n,j}K_{\max}\Delta t}<10^{-6}.
$$

---

Ces quatre récurrences + l’agrégation suffisent pour :

1. **balayer tout l’historique** et obtenir, pour chaque date $t$,
   $R_{1,t}$ et $\sqrt{R_{2,t}}$ ;
2. lancer l’OLS sur le VIX → premier jeu $\beta_0,\beta_1,\beta_2$ ;
3. fournir les valeurs d’amorce $R_{n,j,0}$ nécessaires à la phase 2 (simulation OU avec σ).


### 1.  D’où vient la **formule d’initialisation**

Départ : définition continue du filtre « rapide » (papier, équ. 4.10) :

$$
R_{n,j,t}\;=\;\lambda_{n,j}\!\int_{-\infty}^{t}\!e^{-\lambda_{n,j}(t-u)}
\Bigl(\tfrac{dS_u}{S_u}\Bigr)^{\!n}.
$$

---

**Discrétisation journalière** ($\Delta t=1/252$ an)

$$
\frac{dS_{u}}{S_{u}}\;\longrightarrow\;r_{k}
=\ln S_{t-k\Delta t}-\ln S_{t-(k+1)\Delta t}.
$$

On remplace l’intégrale par une somme :

$$
R_{n,j,t}\;=\;\lambda_{n,j}\!
\sum_{k=1}^{\infty}e^{-\lambda_{n,j}k\Delta t}\,[r_{t-k}]^{\,n}.
\tag{★}
$$

---

**Troncature**
Le poids le plus ancien vaut $e^{-\lambda_{n,j}K_{\max}\Delta t}$.
On choisit $K_{\max}$ tel que ce poids < $10^{-6}$ :

$$
K_{\max}=\Bigl\lceil\frac{6\!+\!\ln\lambda_{n,j}}{\lambda_{n,j}\Delta t}\Bigr\rceil
\approx\Bigl\lceil\frac{7}{\lambda_{n,j}\Delta t}\Bigr\rceil .
$$

On obtient la formule d’initialisation utilisée dans le code :

$$
\boxed{\;
R_{n,j,0}
=\lambda_{n,j}\!\!\sum_{k=1}^{K_{\max}}
e^{-\lambda_{n,j}k\Delta t}\,[r_{-k}]^{\,n}\;}
$$

*(les indices négatifs désignent les $k$ derniers rendements avant « aujourd’hui »).*

---

### 2.  Faut-il « backward puis forward » ?

* **Pas besoin**.
* Utilise simplement la **récurrence directe** dès le plus ancien retour disponible :

$$
R_{n,j}^{\,\text{next}}
=e^{-\lambda_{n,j}\Delta t}\,R_{n,j}^{\,\text{curr}}
+\lambda_{n,j}\,[r_{\text{next}}]^{\,n}.
$$

En un **seul passage vers l’avant**, tu obtiens la série complète des $R_{n,j,t}$ et, à la fin, la valeur $R_{n,j,0}$.

> Le “backward + forward” n’est visible que dans le notebook parce que les auteurs
> calculent toute la série **dans l’ordre chronologique inverse** (`[::-1]`)
> puis la retournent (`[::-1]`) pour l’OLS ; c’est une astuce vectorielle,
> pas une nécessité mathématique.

---

### 3.  Pipeline pratique

1. **Lecture des prix** $\{S_{-T},\dots,S_0\}$.

2. **Boucle unique :**

   ```python
   R10, R11, R20, R21 = 0., 0., 0., 0.
   for r in returns:                      # du plus ancien au plus récent
       R10 = exp(-λ10*dt)*R10 + λ10*r
       R11 = exp(-λ11*dt)*R11 + λ11*r
       R20 = exp(-λ20*dt)*R20 + λ20*r**2
       R21 = exp(-λ21*dt)*R21 + λ21*r**2
   # À la dernière itération on possède R_{n,j,0}
   ```

3. **Agrégation :**
   $R_{1,0}= (1-\theta_1)R_{10}+\theta_1R_{11}$,
   $R_{2,0}= (1-\theta_2)R_{20}+\theta_2R_{21}$.

4. **Régression OLS sur le VIX** → $\beta_0,\beta_1,\beta_2$.

Cette méthode suit exactement la stratégie du papier et du dépôt : un seul balayage « forward » suffit pour calculer **tous** les filtres et produire la valeur initiale des quatre facteurs.


**Non, tu n’es pas “condamné” à calculer $R_{n,j,0}$ par la formule EWMA tronquée avant de dérouler la récursion ; plusieurs stratégies existent.**

---

## Trois façons acceptées d’amorcer les filtres

| Option                                            | Comment on fait                                                                                                                                                                                                                                                                               | Quand c’est suffisant                                                                                                                                        |
| ------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **A. EWMA tronquée** (formule que je t’ai donnée) | $\displaystyle R_{n,j,0}=\lambda_{n,j}\sum_{k=1}^{K_{\max}}e^{-\lambda_{n,j}k\Delta t}r_{-k}^{\,n}$                                                                                                                                                                                           | Tu disposes d’au moins 1–2 ans de quotes et tu veux que $\sigma_0$ colle tout de suite au VIX spot. (C’est l’option du repo/papier)                          |
| **B. “Warm-up” forward only**                     | Fixe $R_{n,j,\text{start}}=0$ ou valeur stationnaire,\* puis fais un seul balayage **vers l’avant** sur l’historique ; la récursion : $R_{n,j,t+\Delta t}=e^{-\lambda\Delta t}R_{n,j,t}+\lambda r_{t+\Delta t}^{\,n}$ fournit tous les $R_{n,j,t}$, y compris la dernière valeur $R_{n,j,0}$. | Tu as l’historique complet ; tu préfères coder simple et tu acceptes un écart minime sur les premiers jours (l’erreur d’amorce décroît en $e^{-\lambda t}$). |
| **C. Valeur latente à calibrer**                  | Traite $R_{n,j,0}$ comme paramètre libre (4 dimensions de plus) et laisse l’optimiseur l’ajuster avec les autres β, λ, θ.                                                                                                                                                                     | Tu n’as quasiment pas d’historique fiable ou tu veux laisser le marché “dire” où placer les filtres.                                                         |

\* Valeur stationnaire simple : $R_{1,j}=0,\; R_{2,j}=\beta_0^2$.

---

### Impact sur la calibration

* **Phase 1 (OLS)**
  Tant que tu disposes d’assez de rendements pour estimer les séries $(VIX_t,R_{1,t},\sqrt{R_{2,t}})$, choisir A ou B ne change quasiment pas la régression : les premiers jours, où la différence est perceptible, pèsent peu sur l’erreur quadratique moyenne.

* **Phase 2 (pricing)**
  L’optimiseur ré-ajuste β, λ (et éventuellement θ, R₀) pour coller à la surface SPX + VIX.
  → Si tu as pris l’option B “warm-up”, la calibration rattrape très vite l’écart initial ; si tu as choisi C, il l’absorbe dans les variables qu’il optimise.

---

### En pratique

* **Repo des auteurs :** option A (EWMA tronquée) pour gagner un démarrage parfaitement cohérent avec le VIX du jour 0.
* **Accepté en production :** option B suffit dans > 90 % des cas ; elle évite de remonter très loin dans le passé et ne nécessite aucun calcul “backward”.

$$
\boxed{\text{Donc\,: tu n’es pas obligé de passer par la somme EWMA complète\,; un simple balayage
forward ou même un R\_0 libre fonctionnent très bien selon tes contraintes.}}
$$


### Phase 1 (“historique”) – bonnes pratiques de statisticien

| Problème courant                                           | Pourquoi ça gêne ?                                                   | Parade simple et robuste                                                                 |      |       |
| ---------------------------------------------------------- | -------------------------------------------------------------------- | ---------------------------------------------------------------------------------------- | ---- | ----- |
| **Qualité des rendements** (dividendes, splits, bad-ticks) | un tick erroné crée un pic sur $r_t$ ⇒ fausse volatilité ⇒ β biaisés | • privilégier séries **total-return** ajustées • winsoriser (                            | r\_t | >6,σ) |
| **Saute-maturité du VIX** (méthode CBOE)                   | VIX spot dérive le lundi / mardi (effet calendrier)                  | appliquer le **VIX-adjacent** (interp. linéaire entre VIX de vendredi et de lundi)       |      |       |
| **Choix de fenêtre $\lambda, θ$**                          | trop lent  ⇒ β absorbe du bruit ; trop rapide ⇒ sur-réactivité       | mini-grille logique (ex : λ ∈ {3,5}, 0.3 / 0.6 an⁻¹) + **validation croisée temporelle** |      |       |
| **Hétéroscédasticité des résidus OLS**                     | β sous-pondère les jours calmes, sur-pondère les chocs               | régression **GLS** pondérée par $1/\operatorname{VIX}_t$ ou quantile regression          |      |       |
| **Autocorrélation des erreurs**                            | l’OLS suppose iid ⇒ variances β sous-estimées                        | Newey-West (lag ≈ 10 jours) pour SE, ou régresser sur **retours hebdo**                  |      |       |
| **Outliers 2020-2022** (COVID, guerre)                     | β surestime les skews longs                                          | robust OLS (Huber/M-estimator) ou couper ±2 % pires résidus                              |      |       |
| **Non-stationnarité lente**                                | β fixe ≠ marché évolutif                                             | *rolling window* 5 ans glissant + moyenne exponentielle des β                            |      |       |
| **Données manquantes (fériés US)**                         | brèche dans récursions $R_{n,j}$                                     | interpoler $r_t=0$ les jours fermés ou estimer sur jours ouvrés seulement                |      |       |
| **Différences de pas (intraday vs daily)**                 | mélange de granularités                                              | tout convertir **daily close** avant filtrage                                            |      |       |

---

#### Pipeline conseillé

1. **Pré-traite** SPX (total-return, filtres bad-ticks) + VIX ajusté calendrier.
2. Calcule $\{r_t\}$ puis filtres $R_{1,j,t},R_{2,j,t}$ avec une grille courte $(\lambda,\theta)$.
3. **OLS robuste** $VIX_t\sim 1,R_{1,t},\sqrt{R_{2,t}}$ ; poids $=1/\text{VIX}_t$.
4. **Cross-validate** : découpe 80 % entraînement / 20 % test ; vérifie erreur RMS < 1 pt de VIX.
5. Conserve le quadruplet $(β,\lambda,\theta)$ le plus stable sur splits temporels.
6. Stocke également les **écarts types robustes** des β pour fixer des bornes dans la phase 2 (évite dérives extrêmes).

---

#### Rappel pratique

* Les filtres oublient l’init < 3 mois quand $λ_{\text{court}}\ge 4$ an⁻¹ : inutile de remonter à l’infini.
* Un simple **balayage forward** suffit pour construire $R_{n,j,t}$ et récupérer $R_{n,j,0}$.
* Garder les mêmes dates SPX/VIX (inner join) pour éviter l’alignement bancal.

En appliquant ces touches “statistiques”, ta phase 1 sera plus robuste et elle donnera un warm-start vraiment solide pour la grande calibration sous $\mathbb Q$.


### Checklist « Phase 1 » — ce qu’il reste à faire et les gardes-fous à poser

*(objectif : sortir un seed robuste pour la calibration Q)*

---

#### 1. Pipeline opérationnel en 5 étapes

1. **Nettoyage data**

   * SPX retours ajustés (splits/dividendes) ; winsoriser $|r_t|>6σ$.
   * VIX spot aligné calendrier (interp. linéaire lundi↔vendredi).

2. **Choix d’une mini-grille mémoire**

   $$
   \lambda_{1,0}\in\{4,6\}\!/\text{an},\quad
   \lambda_{1,1}\in\{0.3,0.6\},\quad
   \lambda_{2,j}=\lambda_{1,j}/2,\quad
   \theta_1,\theta_2\in\{0.15,0.25\}.
   $$

3. **Calcul des filtres** (forward OU ou EWMA)

   $$
   R_{1,j,t}=e^{-\lambda_{1,j}\Delta t}R_{1,j,t-\Delta t}
             +\lambda_{1,j}r_t,\quad
   R_{2,j,t}=e^{-\lambda_{2,j}\Delta t}R_{2,j,t-\Delta t}
             +\lambda_{2,j}r_t^{2}.
   $$

4. **Régression robuste (GLS)**

   $$
   VIX_t = \beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}
   $$

   poids = $1/\text{VIX}_t$ ; SE : Newey-West (lag 10).

5. **Validation temporelle**

   * split 80 % / 20 % ; RMS error test < 1 pt de VIX.
   * garder le quadruplet $(β,\lambda,\theta)$ le plus stable.

---

#### 2.  Contraintes à respecter pendant l’OLS

| Paramètre         | Bornes / logique                                                       | Raison                                        |
| ----------------- | ---------------------------------------------------------------------- | --------------------------------------------- |
| $\lambda_{n,j}$   | $0<\lambda_{2,1}<\lambda_{2,0}<\lambda_{1,1}<\lambda_{1,0}\le 10$ an⁻¹ | ordre « court / long » + stabilité numérique  |
| $\theta_n$        | $0<\theta_n<1$                                                         | pondération convexe                           |
| $\beta_0$         | $>0$                                                                   | garantir $\sigma_t>0$                         |
| $\beta_1$         | généralement $\beta_1<0$                                               | skew négatif empirique                        |
| $\beta_2$         | $0<\beta_2<1$                                                          | éviter explosion des ailes hautes             |
| **Stabilité vol** | $\beta_2^{\,2}\,\theta_2<1$                                            | condition (Guyon §4.2) pour $\sigma_t$ bornée |
| **OU drift**      | $\lambda_{n,j}\,\beta_1^{2}<2$ (suffisant)                             | éviter variance infinie de $R_{1,j}$          |

*(ces inégalités sont faciles à imposer via `bounds=` dans `lstsq` ou lors du grid search).*

---

#### 3.  Sorties à transmettre à la phase Q

* Vecteur **seed**
  $\vartheta^{(0)}=(\beta_0,\beta_1,\beta_2,\lambda_{1,0},\lambda_{1,1},
                   \lambda_{2,0},\lambda_{2,1},\theta_1,\theta_2)$.
* Valeurs d’amorce
  $R_{1,0,0},R_{1,1,0},R_{2,0,0},R_{2,1,0}$ (EWMA sur $K_{\max}$ retours).
* Écarts-types robustes des β pour fixer des bornes « soft » dans l’optimiseur Q.
* Rapport diag : R², MSE, test Ljung-Box des résidus (contrôle autocorr.).

---

#### 4.  Pièges fréquents & remèdes

* **Bad-ticks SPX** → spike sur $r_t$ → winsorisation / filtre Hampel.
* **Brèches calendrier** (Thanksgiving) → combler par $r_t=0$.
* **Outliers COVID 2020** → robuste M-estimator (Huber τ=1.5).
* **Hétéroscédasticité** → GLS pondéré par $1/\text{VIX}$.
* **Peu de données** (< 2 ans) → laisser $R_{n,j,0}$ libres en phase Q.

---

**Next step après cette phase** : passer au moteur pricing (phase 2) avec le seed $\vartheta^{(0)}$, lancer Monte-Carlo + Py-BOBYQA sur surface SPX + VIX tout en gardant les contraintes ci-dessus comme bornes “dures”.


**1. Poids = 1 / VIX t**

* Quand l’erreur de mesure d’un point d’observation croît avec le niveau du VIX (jours très volatils → moins fiables), on pondère chaque ligne de la régression par

  $$
  w_t=\frac{1}{\text{VIX}_t}.
  $$
* Concrètement : on multiplie $X_t$ et $Y_t$ par $\sqrt{w_t}$ avant de lancer l’OLS ; c’est l’équivalent d’une régression généralisée (GLS) avec variance proportionnelle au VIX.

---

**2. SE : Newey-West (lag 10)**

* Les résidus de la régression $VIX_t-\hat VIX_t$ sont autocorrélés (mémoire de quelques jours).
* L’estimateur **Newey-West** corrige les écarts-types des $\beta$ pour hétéroscédasticité **et** autocorrélation jusqu’à un certain décalage ; ici on choisit 10 jours ≈ 2 semaines boursières.
* On obtient ainsi des intervalles de confiance valides sans toucher aux coefficients.

---

**3. `lstsq`**

* Fonction NumPy / PyTorch :

  ```python
  beta, *_ = np.linalg.lstsq(X, y, rcond=None)
  ```

  ou

  ```python
  beta = torch.linalg.lstsq(X, y).solution
  ```
* Résout $\min_\beta \|X\beta - y\|_2^2$ ; pratique pour tester rapidement chaque grille $(\lambda,\theta)$.
* On peut y intégrer les poids : passer $X^\* = \sqrt{w}\,X$ et $y^\* = \sqrt{w}\,y$.

---

**Prochain point : pièges de données (bad-ticks, jours fériés, COVID, etc.) — dis-moi sur lequel tu veux qu’on creuse en premier.**


### Régression « phase 1 » : la configuration recommandée

*(objectif : seed robuste et rapide à calculer)*

| Décision                     | Choix recommandé                                                                                                                            | Pourquoi                                                                                                |
| ---------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------- |
| **Type de régression**       | **GLS pondérée** (*weighted least squares*)                                                                                                 | Donne plus de poids aux points où le VIX est bas (données plus précises)                                |
| **Poids**                    | $w_t = 1/\text{VIX}_t$                                                                                                                      | Variance empirique ≈ proportionnelle au niveau du VIX                                                   |
| **Variables explicatives**   | $X_t = (1,\;R_{1,t},\;\sqrt{R_{2,t}})$                                                                                                      | Forme linéaire exacte du modèle                                                                         |
| **Fenêtre $\lambda,\theta$** | Mini-grille<br> • $\lambda_{\text{court}}\in\{4,6\}$ an⁻¹ <br> • $\lambda_{\text{long}}\in\{0.3,0.6\}$ an⁻¹ <br> • $\theta\in\{0.15,0.25\}$ | Quatre combinaisons faciles à tester ; couvrent les horizons « court » (2-3 mois) et « long » (1-2 ans) |
| **Estimateur**               | `numpy.linalg.lstsq` (ou `torch.linalg.lstsq`) sur $\sqrt{w_t}\,X_t,\sqrt{w_t}\,VIX_t$                                                      | Extrêmement rapide ; suffit pour un grid search                                                         |
| **Contraintes du solveur**   | Appliquer **a posteriori** :<br> • $\beta_0>0$<br> • $\beta_2\in(0,1)$<br> • $\beta_1<0$<br> • $\beta_2^2\theta_2<1$                        | Garantit $\sigma_t>0$ et stabilité du filtre                                                            |
| **Écarts-types β**           | Newey–West (lag = 10 jours) pour SE robustes                                                                                                | Corrige l’autocorrélation des résidus                                                                   |
| **Validation**               | Split 80/20 (temporel) → RMS < 1 pt VIX                                                                                                     | Vérifie la stabilité hors-échantillon                                                                   |

```python
# pseudo-code minimal
w   = 1 / vix.values
X   = np.column_stack([np.ones_like(R1), R1, np.sqrt(R2)])
Xw  = np.sqrt(w)[:,None] * X
yw  = np.sqrt(w) * vix.values
β   = np.linalg.lstsq(Xw, yw, rcond=None)[0]   # β0,β1,β2
```

> Après la régression, **garde** la grille $(\lambda,\theta)$ qui donne l’erreur test la plus basse et dont les β satisfont toutes les contraintes ; c’est ton **seed** pour la calibration pricing.


**Pourquoi pondérer par $1/\text{VIX}_t$ (GLS) est pertinent :**

| Point clé                           | Explication dans le contexte SPX / VIX                                                                                                                                                                                                                                                                                                               |
| ----------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Hétéroscédasticité structurelle** | L’erreur de la relation $VIX_t \approx \beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$ se dilate quand le VIX est élevé : écarts bid-ask plus larges, volatilité intrajournalière plus forte, erreurs de mesure du strip OTM. Sous OLS, ces observations « bruyantes » pèsent autant que les jours calmes ⇒ variances des β gonflées et seed instable. |
| **Robustesse du seed**              | Le but de la phase 1 est un point de départ fiable pour la calibration pricing (phase 2). Le marché vit 80 % du temps dans un régime < 20 % de VIX ; sur-pondérer ces journées améliore la précision là où l’on pricera la plupart des options. Les jours d’extrême stress resteront pris en compte mais ne tirent plus la régression.               |
| **Interprétation statistique**      | Choisir $w_t=1/\text{VIX}_t$ revient à supposer $\operatorname{Var}[\varepsilon_t]\propto\text{VIX}_t$. Test de Breusch-Pagan ou plot résidus² vs VIX confirment généralement cette loi de variance. GLS donne alors les estimateurs BLUE (meilleure variance) sans biais.                                                                           |
| **Alternative**                     | On pourrait garder OLS mais ajuster les SE (Newey-West) ; on obtient des β identiques mais des intervalles larges ⇒ moins de contrôle pour borner la phase 2. Le GLS pondéré réduit directement la variabilité des β.                                                                                                                                |

**En pratique :**

```python
w   = 1 / vix                     # poids inverse du niveau
Xw  = np.sqrt(w)[:,None] * X      # régression pondérée
yw  = np.sqrt(w)       * vix
beta = np.linalg.lstsq(Xw, yw, rcond=None)[0]
```

* Les petites journées de VIX (plus “propres”) guident la pente.
* Les journées à VIX > 40 ne disparaissent pas : elles gardent un poids ≈ 1/40, suffisant pour empêcher un sous-ajustement total des extrêmes.
* On vérifie ensuite RMS hors-échantillon ; si la perte sur les pics de crise devient critique pour ton usage (stress-testing), on peut mélanger **GLS** et **Huber** ou ré-introduire un léger poids plancher.


### Phase 2 : calibration **pricing** (sous 𝑸)

*(version compacte, lisible écran mobile)*

1. **Données fixes**

   * Spot SPX $S_0$ ; courbes taux $r(t)$ et dividendes $q(t)$.
   * Surface IV SPX (1 sem → 3 ans, 5-10 strikes).
   * Strip OTM 30 j (pour VIX), futures VIX 1-6 m, options VIX 1-6 m.

2. **Seed venant de la phase 1**
   $\beta_0,\beta_1,\beta_2,\lambda_{1,0},\lambda_{1,1},
     \lambda_{2,0},\lambda_{2,1},\theta_1,\theta_2$

   * filtres initiaux $R_{1,0,0},R_{1,1,0},R_{2,0,0},R_{2,1,0}$.

3. **Moteur Monte-Carlo (GPU, 5 facteurs)**

   * États : $S,R_{1,0},R_{1,1},R_{2,0},R_{2,1}$.
   * Mise à jour :

     $$
       R_{1,j}^{n+1}=e^{-\lambda_{1,j}\Delta t}\bigl(R_{1,j}^n+\lambda_{1,j}r^{n+1}\bigr),\;
       R_{2,j}^{n+1}=e^{-\lambda_{2,j}\Delta t}\bigl(R_{2,j}^n+\lambda_{2,j}\sigma_n^{2}\Delta t\bigr)
     $$

     avec $r^{n+1}=\sigma_n\Delta W$.
   * 100-200 k chemins, mêmes ΔW (CRN) à chaque itération.

4. **Prix modèle et IV**

   * SPX : moyenne MC → inversion Black-Scholes.
   * VIX futures : VIX théorique (strip ou réseau surrogat).
   * Options VIX : Black-76 + inversion IV.

5. **Fonction coût**

   $$
   J(\vartheta)=\sum w_{\text{SPX}}(\sigma_{\text{mod}}-\sigma_{\text{mkt}})^2
              +\sum w_{\text{VX}}   (F_{\text{mod}}-F_{\text{mkt}})^2
              +\sum w_{\text{VXO}}  (\text{IV}_{\text{mod}}-\text{IV}_{\text{mkt}})^2
              +\gamma\|\vartheta-\vartheta^{(0)}\|^2
   $$

   * Poids $w$= vega / spread.
   * $\gamma\sim10^{-2}$ (attache au seed).

6. **Optimiseur**

   * Py-BOBYQA (sans gradient) **ou** L-BFGS-B + AAD.
   * Bornes dures :
     $\beta_0>0,\;0<\beta_2<1,\;\beta_1<0,\;0<\theta_n<1,\;0.05<\lambda_{long}<\lambda_{short}<10$.

7. **Tests de stabilité en boucle**

   * Rejeter si $\sigma_t<0$ ou >,400 %.
   * Vérifier $\beta_2^2\theta_2<1$.
   * Pénalité forte si violé.

8. **Critère d’arrêt**

   * RMS SPX < 1 % **et** RMS VIX < 3 %
     **ou** amélioration $J<10^{-4}$.

9. **Sorties finales**

   * Paramètres $\vartheta^\star$.
   * Surface IV modélisée, prix VIX, greeks (pathwise/AAD).
   * Rapport erreurs & contrôle stabilité.

> Temps typique : < 5 min sur GPU RTX avec 30-40 itérations.


### Fonction-coût utilisée en phase 2

*(on ajuste tous les paramètres pour que le modèle colle aux marchés SPX + VIX)*

$$
\boxed{
\displaystyle
J(\vartheta)=
\underbrace{\sum_{i}w^{\text{SPX}}_{i}\,\bigl[\sigma^{\text{mod}}_{i}(\vartheta)-\sigma^{\text{mkt}}_{i}\bigr]^2}_{\text{(A) surface SPX}}
\;+\;
\underbrace{\sum_{k}w^{\text{VX}}_{k}\,\bigl[F^{\text{mod}}_{k}(\vartheta)-F^{\text{mkt}}_{k}\bigr]^2}_{\text{(B) futures VIX}}
\;+\;
\underbrace{\sum_{j}w^{\text{VXO}}_{j}\,\bigl[\text{IV}^{\text{mod}}_{j}(\vartheta)-\text{IV}^{\text{mkt}}_{j}\bigr]^2}_{\text{(C) options VIX}}
\;+\;
\underbrace{\gamma\,\lVert\vartheta-\vartheta^{(0)}\rVert^{2}}_{\text{(D) terme de rappel}}
}
$$

---

| Élément                                                                    | Contenu précis                                                                  | Rôle                                                                                                                      |
| -------------------------------------------------------------------------- | ------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------- |
| **Indices $i$**                                                            | chaque point de la **surface d’IV SPX** (maturité × strike)                     | faire reproduire niveau, skew et term-structure SPX                                                                       |
| **$\sigma^{\text{mkt}}_{i}$**                                              | volatilité implicite observée                                                   | cible « marché »                                                                                                          |
| **$\sigma^{\text{mod}}_{i}(\vartheta)$**                                   | IV donnée par le modèle une fois qu’on a prix l’option et inversé Black-Scholes | dépend des paramètres $\vartheta$ qu’on optimise                                                                          |
| **$w^{\text{SPX}}_{i}$**                                                   | $=\dfrac{\text{vega}_{i}}{\text{bid-ask spread}_{i}}$                           | pondère plus fort les options liquides et très sensibles                                                                  |
| **Indices $k$**                                                            | maturités des **futures VIX** (1 à 6 mois)                                      | caler le niveau de la volatilité forward                                                                                  |
| **$F^{\text{mkt}}_{k}$** / $F^{\text{mod}}_{k}(\vartheta)$                 | prix futur VIX observé / modélisé (via strip ou réseau surrogat)                |                                                                                                                           |
| **$w^{\text{VX}}_{k}$**                                                    | $=1/\text{spread}_{k}$ ou $1$ si spreads proches                                |                                                                                                                           |
| **Indices $j$**                                                            | strikes-maturités des **options VIX**                                           | reproduire le smile du VIX                                                                                                |
| **$\text{IV}^{\text{mkt}}_{j}$** / $\text{IV}^{\text{mod}}_{j}(\vartheta)$ | IV VIX marché / modèle (Black-76)                                               |                                                                                                                           |
| **$w^{\text{VXO}}_{j}$**                                                   | vega / spread (analogue à SPX)                                                  |                                                                                                                           |
| **$\gamma$**                                                               | poids de régularisation (∼ $10^{-2}$)                                           | évite que les paramètres s’éloignent trop du **seed** historique $\vartheta^{(0)}$ ; stabilise et accélère la convergence |
| **$\lVert\vartheta-\vartheta^{(0)}\rVert^{2}$**                            | distance euclidienne entre paramètres courants et seed                          | pénalisation douce (« tether »)                                                                                           |

---

#### Lecture rapide

* **Termes (A), (B), (C)** : erreurs quadratiques entre le modèle et le marché sur :

  * surface d’options SPX,
  * futures VIX,
  * options VIX.
    Chaque écart est **pondéré** par un poids qui reflète la liquidité (vega) et la confiance (spread).
* **Terme (D)** : régularisation qui maintient les nouveaux paramètres proches des valeurs historiques, sauf si la surface d’options « demande » autre chose.

Minimiser $J(\vartheta)$ fait donc chercher l’équilibre optimal entre reproduire fidèlement **toutes** les quotes du marché et rester économiquement raisonnable vis-à-vis du seed issu de la phase 1.


### Trois « sigmas » à ne pas confondre

| Symbole                       | Que c’est ?                                                                                          | Comment on l’obtient                                              | Où il apparaît                                         |
| ----------------------------- | ---------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------- | ------------------------------------------------------ |
| **$\sigma_t$**                | **Volatilité instantanée** du modèle (formule $\beta_0+\beta_1R_{1,t}+\beta_2\sqrt{R_{2,t}}$).       | Directement dans la simulation Monte-Carlo à chaque pas de temps. | SDE du sous-jacent, mise à jour des filtres $R_{n,j}$. |
| **$\sigma^{\text{mod}}_{i}$** | **Implied vol** que le **même modèle** donne pour une **option SPX** de strike $K_i$ maturité $T_i$. | 1. On simule le prix de l’option avec le moteur MC/PDE.           |                                                        |

2. On **inverse Black-Scholes** (sous-jacent = SPX) pour trouver la volatilité qui reproduit ce prix. | Terme (A) de la fonction-coût : surface SPX. |
   \| **$\text{IV}^{\text{mod}}_{j}$** | **Implied vol** que le **modèle** donne pour une **option VIX** (futures sous-jacent) de strike $k_j$ maturité $\tau_j$. | 1. Le modèle fournit le prix de l’option VIX (Black-76, sous-jacent = futur VIX).
3. On **inverse Black-76** pour obtenir l’« IV VIX ». | Terme (C) de la fonction-coût : smile VIX. |

> Les deux derniers sont **tous les deux** des implied volatilités (des « sigmas de marché »), simplement :
>
> * $\sigma^{\text{mod}}_{i}$ → options **SPX** ;
> * $\text{IV}^{\text{mod}}_{j}$ → options **VIX**.
>   On leur donne des noms différents pour bien rappeler la formule d’inversion utilisée (Black-Scholes vs Black-76) et le sous-jacent concerné.

### Pourquoi on ne met pas directement $\sigma_t$ dans la fonction-coût ?

* $\sigma_t$ est **instantané** et path-dépendant ; une option, elle, dépend de l’**espérance** de toute la trajectoire future de $\sigma$.
* Le marché cote uniquement des **prix** ou des **implied vols**.
  *Pour pouvoir comparer modèle ↔ marché*, on transforme toujours le prix modèle en la **même métrique** que le marché : l’implied volatility (SPX ou VIX).

### Résumé éclair

* **$\sigma_t$** → vol « microscopique » du modèle.
* **$\sigma^{\text{mod}}_{i}$** → IV SPX (modèle).
* **$\text{IV}^{\text{mod}}_{j}$** → IV VIX (modèle).

La fonction-coût compare les deux derniers aux **IV du marché**, jamais $\sigma_t$.


### Où trouve-t-on les **options VIX** côté marché ?

| Source                                  | Contenu                                                                                                                                 | Comment on l’utiliser                                                |
| --------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------- |
| **CBOE VX options** (symbole : *“VIX”*) | Bid/ask, volume, IV mid pour :<br>• maturités mensuelles (le mercredi 30 j avant VX future)<br>• strikes env. 80 % – 120 % du VIX futur | Télécharger chaque soir (FTP ou API) → IV $_{\text{mkt}}$ et spreads |
| **VX futures** (symbole : *VX1…VX13*)   | Prix du sous-jacent requis par Black-76                                                                                                 | même fichier de la CBOE                                              |

---

### Comment le **modèle** calcule un prix d’option VIX

1. **Simuler jusqu’à l’échéance de l’option**

   * Chemins 5-facteurs $(S,R_{1,0},R_{1,1},R_{2,0},R_{2,1})$ jusqu’au temps $\tau$.

2. **Obtenir le niveau du VIX à $\tau$**

   * Dans le papier, il existe une **formule fermée** :

     $$
     \text{VIX}_\tau^{2}
     =a_1(\tau)\,R_{2,0,\tau}+a_2(\tau)\,R_{2,1,\tau}
     \quad\text{(≈ intégrale future de }σ_t^{2}\text{)}.
     $$
   * Dans le repo : ils remplacent cette formule par un **réseau neurone surrogat** $f_{\text{NN}}(R_\tau)$ entraîné une fois pour toutes → plus de « nested MC ».

3. **Payer le payoff**

   $$
   \text{Payoff}=\max\bigl(\text{VIX}_\tau-K,\,0\bigr)\quad(\text{call}),\qquad
   \text{ou } \max(K-\text{VIX}_\tau,0)\quad(\text{put}).
   $$

4. **Actualiser**
   $\displaystyle C^{\text{mod}}=e^{-r\tau}\,\frac1N\sum_{\text{paths}}\text{Payoff}$.

5. **Transforme en implied vol VIX**

   * Sous-jacent = futur VIX $F_\tau^{\text{mod}}$ (moyenne de $\text{VIX}_\tau$).
   * On applique **Black-76** : on cherche $\sigma_{\text{BS}}$ tel que
     $C_{\text{BS}}(F_\tau^{\text{mod}},K,\tau,\sigma_{\text{BS}})=C^{\text{mod}}$.
   * Cette $\sigma_{\text{BS}}$ devient $\text{IV}^{\text{mod}}_{j}$ pour la fonction-coût.

---

### Pourquoi pas « directement σ » ?

* Le marché **cote** une IV VIX (Black-76), pas le prix brut ni la volatilité instantanée $\sigma_t$.
* Pour comparer « pomme à pomme », on convertit toujours *prix modèle* → *même métrique* que le marché.

---

### Résumé pipeline « option VIX » côté modèle

```
simulate paths 0→τ
      ↓
VIX_τ  = a1*R20_τ + a2*R21_τ   (ou NN surrogate)
      ↓
discount payoff max(VIX_τ-K,0)
      ↓
price C_mod
      ↓
invert Black-76  →  IV_mod
```

Les **IV$_{\text{mod}}$** ainsi obtenues sont ensuite comparées aux **IV$_{\text{mkt}}$** dans le terme (C) de la fonction-coût.
